In [1]:
from striped.job import SinglePointStripedSession as Session, IPythonDisplay

from histbook import Hist, beside, groupby, below
from histbook import bin as hbin
import re
import uproot
import numpy



corrDict = {}

#print di

correctionFiles = open("correctionFiles.txt").readlines()



for f in correctionFiles:
  
    dumFile = uproot.open("correction_files/"+f.strip())
    for i, key in enumerate(dumFile.keys()):
        TH1D = "<class 'uproot.rootio.TH1D'>"
        TH2D = "<class 'uproot.rootio.TH2D'>"
        TH1F = "<class 'uproot.rootio.TH1F'>"
        TH2F = "<class 'uproot.rootio.TH2F'>"
        RootDir = "<class 'uproot.rootio.ROOTDirectory'>"
        histType = str(type(dumFile[key[:-2]]))
        if histType == TH1D or histType == TH2D or histType==TH1F or histType==TH2F:
            if  not ("bound method" in str(dumFile[key[:-2]].numpy)):
                corrDict[f[:-6]+"_"+dumFile.keys()[i][:-2]] = dumFile[key[:-2]].numpy
            else:
                corrDict[f[:-6]+"_"+dumFile.keys()[i][:-2]] = dumFile[key[:-2]].numpy()
                
                
        elif histType == RootDir:
               for j, key2 in enumerate(dumFile[key[:-2]].keys()):
                    histType2 = str(type(dumFile[key[:-2]][key2[:-2]]))
                    if histType2 == TH1D or histType2 == TH2D or histType2==TH1F or histTyp2==TH2F:
                        if not ("bound method" in str(dumFile[key[:-2]][key2[:-2]].numpy)):
                            corrDict[f[:-6]+"_"+key[:-2]+"_"+key2[:-2]] = dumFile[key[:-2]][key2[:-2]].numpy
                        else:
                            corrDict[f[:-6]+"_"+key[:-2]+"_"+key2[:-2]] = dumFile[key[:-2]][key2[:-2]].numpy()
                            
                            
        else:
         #   print f+ " -> " +key[:-2]
            tempArrX= dumFile[key[:-2]]._fEX
            tempArrY= dumFile[key[:-2]]._fEY
            corrDict[key[:-2]] = [tempArrX, tempArrY]
          #  print corrDict[key[:-2]]
    
#print corrDict
'''
for  i in range(0,len(corrDict)):
    if not isinstance(corrDict.items()[i][1][1][0], numpy.float64):
        print corrDict.items()[i][0]
 '''    


def valueAt(dictionary ,key,x,y = None):
    if y ==None:
        if not isinstance(dictionary[key][1][0], numpy.float64):
            print "dictionary[" +key+"] is  2D"
            return 'NaN'
        xBinMin = dictionary[key][1][0]
        xBinMax = dictionary[key][1][len(dictionary[key][1])-1]
        if x<xBinMin or x> xBinMax:
            print "point out  of range" + " ("+str(xBinMin) + ","+ str(xBinMax) +")"
            return 'NaN'
        i = 0

        xBin = dictionary[key][1][0]


        while(x>xBin):
            i = i +1
            xBin = dictionary[key][1][i]

        i = i-1

        return dictionary[key][0][i]
    else:
        if isinstance(dictionary[key][1][0], numpy.float64):
            print "dictionary[" +key+"] is not 2D"
            return 'NaN'
        xBinMin = dictionary[key][1][0][0]
        xBinMax = dictionary[key][1][0][len(dictionary[key][1][0])-1]
        yBinMin = dictionary[key][1][1][0]
        yBinMax = dictionary[key][1][1][len(dictionary[key][1][1])-1]
        if x<xBinMin or x> xBinMax or y < yBinMin or y > yBinMax:
            print "point out  of range x:("+str(xBinMin)+","+str(xBinMax)+") y:("+str(yBinMin)+","+str(yBinMax) +")"
            return 'NaN'
        i = 0
        j = 0
        xBin = dictionary[key][1][0][0]
        yBin = dictionary[key][1][1][0]

        while(x>xBin):
            i = i +1
            xBin = dictionary[key][1][0][i]
        while(y>yBin):
            j = j +1
            yBin = dictionary[key][1][1][j]
        i = i-1
        j = j-1
        return dictionary[key][0][i][j]

print valueAt(corrDict,"muon_scalefactors_37ifb_scalefactors_TightId_Muon", 55)
'''
#sf_pu
sf_pu = valueAt(corDict,"puWeights_80x_37ifb_puWeights",nvpTrue)
#sf_tt
sf_tt
#sf_lepID
    #if muon
        #if tight
            #if complicated leptons
                        sf_lepID = valueAt(corrDict,"leptonic/muon_scalefactors_37ifb_scalefactors_TightId_Muon",abs(event.muons.eta(),event.muons.pt)
            #if not complicated leptons
                        sf_lepID = valueAt(corrDict,"moriond17/muon_scalefactors_37ifb_scalefactors_TightId_Muon",abs(event.muons.eta(),event.muons.pt)

        #if loose
            #if complicated leptons
                        sf_lepID = valueAt(corrDict,"leptonic/muon_scalefactors_37ifb_scalefactors_MuonLooseId_Muon",abs(event.muons.eta()),event.muons.pt
            #if not complicated lepotns
                        sf_lepID = valueAt(corrDict,"moriond17/muon_scalefactors_37ifb_scalefactors_MuonLooseId_Muon",abs(event.muons.eta()),event.muons.pt
     #if electron
        #if tight
                #if complicated leptons
                    sf_lepID = valueAt(corrDict,"leptonic/scalefactors_80x_egpog_37ifb.root","scalefactors_Tight_Electron",event.electrons.eta(),event.electrons.pt)
                #if not complicated leptons
                     sf_lepID = valueAt(corrDict,"moriond17/scaleFactor_electron_summer16.root","scaleFactor_electron_tightid_RooCMSShape_pu_0_100",event.electrons.eta(),event.electrons.pt)
        # if loose
                sf_lepID = valueAt(corrDict,"moriond17/scaleFactor_electron_summer16.root","scaleFactor_electron_vetoid_RooCMSShape_pu_0_100",event.electrons.eta(),event.electrons.pt)
#sf_lepISO
    # if muon
        #if tight
             #if complicated leptons
                  sf_lepIso = valueAt(corrDict,"leptonic/muon_scalefactors_37ifb.root","scalefactors_Iso_MuonTightId",abs(event.muons.eta(),event.muons.pt)
            #if not complicated leptons
                   sf_lepIso = valueAt(corrDict,"moriond17/muon_scalefactors_37ifb.root","scalefactors_Iso_MuonTightId",abs(event.muons.eta(),event.muons.pt)
        #if loose
            #if complicated leptons       
                              sf_lepIso = valueAt(corrDict,"leptonic/muon_scalefactors_37ifb.root","scalefactors_Iso_MuonLooseId",abs(event.muons.eta(),event.muons.pt)
            # if not complicated leptons
                            sf_lepIso = valueAt(corrDict,"moriond17/muon_scalefactors_37ifb.root","scalefactors_Iso_MuonLooseId",abs(event.muons.eta(),event.muons.pt)
                      
#sf_leptTrack   
        #if muon
               #if complicated lepton
          sf_lepTrack = valueAt(corrDict,"leptonic/Tracking_EfficienciesAndSF_BCDEFGH.root","ratio_eff_eta3_dr030e030_corr",event.npv)
                #if not complicated lepton
         sf_lepTrack = valueAt(corrDict,"moriond17/Tracking_EfficienciesAndSF_BCDEFGH.root","ratio_eff_eta3_dr030e030_corr",event.npv)
             #if electron
            #if complicated leptons
                sf_leptTrack =   valueAt(corrDict,"leptonic/scalefactors_80x_egpog_37ifb.root","scalefactors_Reco_Electron",event.electrons.eta.event.electrons.pt)      
                #if not complicated leptons
                                   sf_lepTrack = valueAt(corrDict,"moriond17/scaleFactor_electron_reco_summer16.root","scaleFactor_electron_reco_RooCMSShape_pu_0_100",event.electrons.eta, event.electorns.pt)
                                                
#sf_ewkV
                                                
                                                
                                                
''' 
def csvToVec(csvFilePath):
    p = csvFilePath
    f = open(p).readlines()
    
    retVec = []
    for line in f:
        line = re.sub(',', '', line)
        vec = line.split()
        retVec.append(vec)
    retVec.pop(0)
    for i, e in enumerate(retVec):
        retVec[i][0]= int(e[0])
        retVec[i][3]=int(e[3])
        retVec[i][4]=float(e[4])
        retVec[i][5]=float(e[5])
        retVec[i][6]=float(e[6])
        retVec[i][7]=float(e[7])
        retVec[i][8]=float(e[8])
        retVec[i][9]=float(e[9])
        retVec[i][10]=eval(e[10])
        
    return retVec

def makeSfSpecificVec(csvVec,OP, mType,sysType, flavor):
    OPVec = []
    for e in csvVec:
        if e[0] == OP and e[1] ==mType and e[2]==sysType and e[3] == flavor:
            e.pop(0)
            e.pop(0)
            e.pop(0)
            e.pop(0)
            OPVec.append(e)
    
    return OPVec

def calcSFBtag(sfCalcVec, eta, pt):
    ret = 0
    if pt < sfCalcVec[0][2]:
        pt = sfCalcVec[0][2]+0.0001
    if pt > sfCalcVec[len(sfCalcVec)-1][3]:
        pt = sfCalcVec[len(sfCalcVec)-1][3] -0.0001
    for e in sfCalcVec:
        if pt > e[2] and pt < e[3]:
            x = pt
            ret = eval(e[6])
            return ret
if 1 == 1.0:
    print  "yes"
    
csvVec = csvToVec("CSVv2_Moriond17_B_H.csv")
OPVec = makeSfSpecificVec(csvVec,0,'mujets','down',0)
#print OPVec
print calcSFBtag(OPVec, 2.1,25)
x = 1
s2 = "0.884016*((1.+(0.0331508*x))/(1.+(0.0285096*x)))"
s = str(csvToVec("CSVv2_Moriond17_B_H.csv")[0][10])
b = eval("0.884016*((1.+(0.0331508*x))/(1.+(0.0285096*x)))")

#print corrDict[corrDict.items()[0][0]][1]
job_server = ("ifdb01.fnal.gov", 8765)

dictionary[muon_scalefactors_37ifb_scalefactors_TightId_Muon] is  2D
NaN
yes
0.918461111714


In [2]:
bg_datasets = """
DYJetsToLL_M-50_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8
DYJetsToNuNu_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8
WJetsToLNu_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8
ZJetsToNuNu_HT-100To200_13TeV-madgraph
ZJetsToNuNu_HT-200To400_13TeV-madgraph
ZJetsToNuNu_HT-400To600_13TeV-madgraph
ZJetsToNuNu_HT-600To800_13TeV-madgraph
ZJetsToNuNu_HT-800To1200_13TeV-madgraph
ZJetsToNuNu_HT-1200To2500_13TeV-madgraph
ZJetsToNuNu_HT-2500ToInf_13TeV-madgraph
ZJetsToNuNu_HT-600ToInf_13TeV-madgraph
DYJetsToLL_M-50_HT-70to100_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
DYJetsToLL_M-50_HT-100to200_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
DYJetsToLL_M-50_HT-200to400_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
DYJetsToLL_M-50_HT-400to600_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
DYJetsToLL_M-50_HT-600toInf_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
DYJetsToLL_M-50_HT-1200to2500_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
DYJetsToLL_M-50_HT-2500toInf_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
DYJetsToLL_M-50_HT-600to800_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
DYJetsToLL_M-50_HT-800to1200_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
WJetsToLNu_HT-70To100_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
WJetsToLNu_HT-100To200_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
WJetsToLNu_HT-200To400_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
WJetsToLNu_HT-400To600_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
WJetsToLNu_HT-600To800_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
WJetsToLNu_HT-800To1200_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
WJetsToLNu_HT-1200To2500_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
WJetsToLNu_HT-2500ToInf_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
WJetsToLNu_HT-600ToInf_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
WJetsToLNu_Pt-100To250_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8
WJetsToLNu_Pt-250To400_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8
WJetsToLNu_Pt-400To600_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8
WJetsToLNu_Pt-600ToInf_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8
DYJetsToLL_Pt-50To100_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8
DYJetsToLL_Pt-100To250_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8
DYJetsToLL_Pt-250To400_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8
DYJetsToLL_Pt-400To650_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8
DYJetsToLL_Pt-650ToInf_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8
DYJetsToNuNu_PtZ-100To250_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8
DYJetsToNuNu_PtZ-250To400_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8
DYJetsToNuNu_PtZ-400To650_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8
DYJetsToNuNu_PtZ-650ToInf_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8
GJets_HT-40To100_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
GJets_HT-100To200_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
GJets_HT-200To400_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
GJets_HT-400To600_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
GJets_HT-600ToInf_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
QCD_HT50to100_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
QCD_HT100to200_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
QCD_HT200to300_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
QCD_HT300to500_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
QCD_HT500to700_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
QCD_HT700to1000_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
QCD_HT1000to1500_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
QCD_HT1500to2000_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
QCD_HT2000toInf_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
ST_t-channel_antitop_4f_leptonDecays_13TeV-powheg-pythia8
ST_t-channel_top_4f_leptonDecays_13TeV-powheg-pythia8
ST_tW_top_5f_inclusiveDecays_13TeV-powheg-pythia8_TuneCUETP8M1
ST_tW_antitop_5f_inclusiveDecays_13TeV-powheg-pythia8_TuneCUETP8M1
ST_t-channel_antitop_4f_inclusiveDecays_13TeV-powhegV2-madspin-pythia8_TuneCUETP8M1
ST_t-channel_top_4f_inclusiveDecays_13TeV-powhegV2-madspin-pythia8_TuneCUETP8M1
ST_s-channel_4f_leptonDecays_13TeV-amcatnlo-pythia8_TuneCUETP8M1
ST_t-channel_5f_leptonDecays_13TeV-amcatnlo-pythia8_TuneCUETP8M1
ST_t-channel_top_4f_leptonDecays_13TeV-powheg-pythia8_TuneCUETP8M1
TTJets_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
TTJets_DiLept_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
TTJets_SingleLeptFromT_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
TTJets_SingleLeptFromTbar_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
TTJets_SingleLeptFromT_TuneCUETP8M2T4_13TeV-amcatnloFXFX-pythia8
TTJets_SingleLeptFromTbar_TuneCUETP8M2T4_13TeV-amcatnloFXFX-pythia8
TT_TuneEE5C_13TeV-powheg-herwigpp
TTJets_TuneCUETP8M2T4_13TeV-amcatnloFXFX-pythia8
TT_TuneCUETP8M2T4_13TeV-powheg-pythia8
TT_TuneCUETP8M2T4_13TeV-powheg-isrdown-pythia8
TT_TuneCUETP8M2T4_13TeV-powheg-isrup-pythia8
TT_TuneCUETP8M2T4down_13TeV-powheg-pythia8
TT_TuneCUETP8M2T4up_13TeV-powheg-pythia8
TTTo2L2Nu_TuneCUETP8M2_ttHtranche3_13TeV-powheg-pythia8
tZq_ll_4f_13TeV-amcatnlo-pythia8
tZq_nunu_4f_13TeV-amcatnlo-pythia8_TuneCUETP8M1
TTGJets_TuneCUETP8M1_13TeV-amcatnloFXFX-madspin-pythia8
TGJets_TuneCUETP8M1_13TeV_amcatnlo_madspin_pythia8
EWKZ2Jets_ZToNuNu_13TeV-madgraph-pythia8
EWKZ2Jets_ZToLL_M-50_13TeV-madgraph-pythia8
EWKWPlus2Jets_WToLNu_M-50_13TeV-madgraph-pythia8
EWKWMinus2Jets_WToLNu_M-50_13TeV-madgraph-pythia8
WW_TuneCUETP8M1_13TeV-pythia8
WZ_TuneCUETP8M1_13TeV-pythia8
ZZ_TuneCUETP8M1_13TeV-pythia8
WWTo2L2Nu_13TeV-powheg
WWTo4Q_13TeV-powheg
WWToLNuQQ_13TeV-powheg
WZTo1L1Nu2Q_13TeV_amcatnloFXFX_madspin_pythia8
WZTo1L3Nu_13TeV_amcatnloFXFX_madspin_pythia8
WZTo2L2Q_13TeV_amcatnloFXFX_madspin_pythia8
WZTo3LNu_TuneCUETP8M1_13TeV-powheg-pythia8
ZZTo2L2Nu_13TeV_powheg_pythia8
ZZTo2L2Q_13TeV_amcatnloFXFX_madspin_pythia8
ZZTo4L_13TeV_powheg_pythia8
GluGluToContinToZZTo2e2nu_13TeV_MCFM701_pythia8
GluGluToContinToZZTo2mu2nu_13TeV_MCFM701_pythia8
GluGluToContinToZZTo2e2mu_13TeV_MCFM701_pythia8
GluGluToContinToZZTo2e2tau_13TeV_MCFM701_pythia8
GluGluToContinToZZTo2mu2tau_13TeV_MCFM701_pythia8
GluGluToContinToZZTo4e_13TeV_MCFM701_pythia8
GluGluToContinToZZTo4mu_13TeV_MCFM701_pythia8
GluGluToContinToZZTo4tau_13TeV_MCFM701_pythia8
ZH_HToBB_ZToNuNu_M125_13TeV_powheg_pythia8
ZH_HToBB_ZToNuNu_M125_13TeV_powheg_pythia8
ZH_HToBB_ZToLL_M125_13TeV_powheg_pythia8
ZH_HToBB_ZToQQ_M125_13TeV_powheg_pythia8
ggZH_HToBB_ZToNuNu_M125_13TeV_powheg_pythia8
ggZH_HToBB_ZToLL_M125_13TeV_powheg_pythia8
WminusH_HToBB_WToLNu_M125_13TeV_powheg_pythia8
WplusH_HToBB_WToLNu_M125_13TeV_powheg_pythia8
WplusH_HToBB_WToQQ_M125_13TeV_powheg_pythia8
WminusH_HToBB_WToQQ_M125_13TeV_powheg_pythia8
WWTo4Q_4f_13TeV_amcatnloFXFX_madspin_pythia8
WJetsToQQ_HT180_13TeV-madgraphMLM-pythia8
ZZTo4Q_13TeV_amcatnloFXFX_madspin_pythia8
DYJetsToQQ_HT180_13TeV-madgraphMLM-pythia8
ttHTobb_M125_13TeV_powheg_pythia8
ttHTobb_M125_TuneCUETP8M2_ttHtranche3_13TeV-powheg-pythia8
GluGluHToBB_M125_13TeV_powheg_pythia8
VBFHToBB_M125_13TeV_amcatnlo_pythia8
VBFHToBB_M-125_13TeV_powheg_pythia8_weightfix
""".split()
signal_samples = """
TTbarDMJets_pseudoscalar_Mchi-1_Mphi-100_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
thq
thw
GluGlu_HToInvisible_M125_13TeV_powheg_pythia8
VBF_HToInvisible_M125_13TeV_powheg_pythia8
VBF_HToInvisible_M110_13TeV_powheg_pythia8
VBF_HToInvisible_M150_13TeV_powheg_pythia8
VBF_HToInvisible_M200_13TeV_powheg_pythia8
VBF_HToInvisible_M300_13TeV_powheg_pythia8
VBF_HToInvisible_M500_13TeV_powheg_pythia8
VBF_HToInvisible_M600_13TeV_powheg_pythia8
VBF_HToInvisible_M800_13TeV_powheg_pythia8
VBF_HToInvisible_M1000_13TeV_powheg_pythia8
GluGlu_HToInvisible_M110_13TeV_powheg_pythia8
GluGlu_HToInvisible_M150_13TeV_powheg_pythia8
GluGlu_HToInvisible_M200_13TeV_powheg_pythia8
GluGlu_HToInvisible_M300_13TeV_powheg_pythia8
GluGlu_HToInvisible_M500_13TeV_powheg_pythia8
GluGlu_HToInvisible_M600_13TeV_powheg_pythia8
GluGlu_HToInvisible_M800_13TeV_powheg_pythia8
GluGlu_HToInvisible_M1000_13TeV_powheg_pythia8
ZprimeToTTJet_M-500_TuneCUETP8M1_13TeV-amcatnlo-pythia8
ZprimeToTTJet_M-750_TuneCUETP8M1_13TeV-amcatnlo-pythia8
ZprimeToTTJet_M-1000_TuneCUETP8M1_13TeV-amcatnlo-pythia8
ZprimeToTTJet_M-1250_TuneCUETP8M1_13TeV-amcatnlo-pythia8
ZprimeToTTJet_M-1500_TuneCUETP8M1_13TeV-amcatnlo-pythia8
ZprimeToTTJet_M-2000_TuneCUETP8M1_13TeV-amcatnlo-pythia8
ZprimeToTTJet_M-2500_TuneCUETP8M1_13TeV-amcatnlo-pythia8
ZprimeToTTJet_M-3000_TuneCUETP8M1_13TeV-amcatnlo-pythia8
ZprimeToTTJet_M-3500_TuneCUETP8M1_13TeV-amcatnlo-pythia8
ZprimeToTTJet_M-4000_TuneCUETP8M1_13TeV-amcatnlo-pythia8
ZprimeToTT_M-1000_W-100_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
ZprimeToTT_M-1000_W-10_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
ZprimeToTT_M-1000_W-300_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
ZprimeToTT_M-1250_W-125_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
ZprimeToTT_M-1250_W-12p5_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
ZprimeToTT_M-1500_W-150_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
ZprimeToTT_M-1500_W-15_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
ZprimeToTT_M-2000_W-200_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
ZprimeToTT_M-2000_W-20_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
ZprimeToTT_M-2000_W-600_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
ZprimeToTT_M-2500_W-250_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
ZprimeToTT_M-2500_W-25_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
ZprimeToTT_M-3000_W-300_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
ZprimeToTT_M-3000_W-30_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
ZprimeToTT_M-3500_W-350_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
ZprimeToTT_M-3500_W-35_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
ZprimeToTT_M-4000_W-1200_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
ZprimeToTT_M-4000_W-400_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
ZprimeToTT_M-4000_W-40_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
ZprimeToTT_M-4500_W-450_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
ZprimeToTT_M-4500_W-45_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
ZprimeToTT_M-5000_W-1500_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
ZprimeToTT_M-5000_W-500_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
ZprimeToTT_M-5000_W-50_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
ZprimeToTT_M-500_W-50_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
ZprimeToTT_M-500_W-5_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
ZprimeToTT_M-750_W-75_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
ZprimeToTT_M-750_W-7p5_TuneCUETP8M1_13TeV-madgraphMLM-pythia8
thq
thw
ZprimeToWW_narrow_M-800_13TeV-madgraph
ZprimeToWW_narrow_M-1000_13TeV-madgraph
ZprimeToWW_narrow_M-1200_13TeV-madgraph
ZprimeToWW_narrow_M-1400_13TeV-madgraph
ZprimeToWW_narrow_M-1600_13TeV-madgraph
ZprimeToWW_narrow_M-1800_13TeV-madgraph
ZprimeToWW_narrow_M-2000_13TeV-madgraph
ZprimeToWW_narrow_M-2500_13TeV-madgraph
ST_tch_DM-scalar_LO-100_1-13_TeV
ST_tch_DM-scalar_LO-300_1-13_TeV
ST_tch_DM-scalar_LO-500_1-13_TeV
ST_tch_DM-scalar_LO-1000_1-13_TeV
TT_DM-scalar_LO-300_1-13_TeV
TT_DM-scalar_LO-500_1-13_TeV
TT_DM-scalar_LO-1000_1-13_TeV
MonoHbb_ZpBaryonic_MZp-10_MChi-1_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-10_MChi-10_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-10_MChi-50_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-10_MChi-150_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-10_MChi-500_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-10_MChi-1000_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-15_MChi-10_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-20_MChi-1_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-50_MChi-1_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-50_MChi-10_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-50_MChi-50_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-95_MChi-50_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-100_MChi-1_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-100_MChi-10_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-200_MChi-1_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-200_MChi-50_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-200_MChi-150_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-295_MChi-150_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-300_MChi-1_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-300_MChi-50_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-500_MChi-1_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-500_MChi-150_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-500_MChi-500_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-995_MChi-500_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-1000_MChi-1_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-1000_MChi-150_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-1000_MChi-1000_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-1995_MChi-1000_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-2000_MChi-1_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-2000_MChi-500_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-10000_MChi-1_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-10000_MChi-10_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-10000_MChi-50_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-10000_MChi-150_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-10000_MChi-500_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-10000_MChi-1000_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-800_MA0-600_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-800_MA0-500_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-800_MA0-400_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-800_MA0-300_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-600_MA0-400_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-600_MA0-300_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-2500_MA0-800_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-2500_MA0-700_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-2500_MA0-600_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-2500_MA0-500_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-2500_MA0-400_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-2500_MA0-300_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-2000_MA0-800_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-2000_MA0-700_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-2000_MA0-600_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-2000_MA0-500_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-2000_MA0-400_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-2000_MA0-300_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1700_MA0-800_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1700_MA0-700_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1700_MA0-600_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1700_MA0-500_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1700_MA0-400_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1700_MA0-300_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1400_MA0-800_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1400_MA0-700_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1400_MA0-600_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1400_MA0-500_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1400_MA0-400_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1400_MA0-300_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1200_MA0-800_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1200_MA0-700_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1200_MA0-600_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1200_MA0-500_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1200_MA0-400_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1200_MA0-300_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1000_MA0-800_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1000_MA0-700_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1000_MA0-600_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1000_MA0-500_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1000_MA0-400_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1000_MA0-300_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-10_MChi-1_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-10_MChi-10_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-10_MChi-50_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-10_MChi-150_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-10_MChi-500_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-10_MChi-1000_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-15_MChi-10_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-20_MChi-1_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-50_MChi-1_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-50_MChi-10_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-50_MChi-50_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-95_MChi-50_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-100_MChi-1_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-100_MChi-10_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-200_MChi-1_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-200_MChi-50_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-200_MChi-150_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-295_MChi-150_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-300_MChi-1_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-300_MChi-50_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-500_MChi-1_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-500_MChi-150_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-500_MChi-500_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-995_MChi-500_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-1000_MChi-1_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-1000_MChi-150_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-1000_MChi-1000_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-1995_MChi-1000_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-2000_MChi-1_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-2000_MChi-500_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-10000_MChi-1_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-10000_MChi-10_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-10000_MChi-50_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-10000_MChi-150_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-10000_MChi-500_13TeV-madgraph
MonoHbb_ZpBaryonic_MZp-10000_MChi-1000_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-2750_MA0-300_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-2750_MA0-500_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-2750_MA0-800_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-3000_MA0-300_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-3000_MA0-500_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-3000_MA0-800_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-3500_MA0-300_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-3500_MA0-500_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-3500_MA0-800_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-4000_MA0-300_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-4000_MA0-500_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-4000_MA0-800_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-800_MA0-600_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-800_MA0-500_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-800_MA0-400_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-800_MA0-300_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-600_MA0-400_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-600_MA0-300_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-2500_MA0-800_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-2500_MA0-700_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-2500_MA0-600_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-2500_MA0-500_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-2500_MA0-400_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-2500_MA0-300_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-2000_MA0-800_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-2000_MA0-700_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-2000_MA0-600_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-2000_MA0-500_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-2000_MA0-400_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-2000_MA0-300_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1700_MA0-800_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1700_MA0-700_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1700_MA0-600_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1700_MA0-500_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1700_MA0-400_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1700_MA0-300_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1400_MA0-800_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1400_MA0-700_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1400_MA0-600_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1400_MA0-500_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1400_MA0-400_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1400_MA0-300_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1200_MA0-800_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1200_MA0-700_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1200_MA0-600_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1200_MA0-500_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1200_MA0-400_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1200_MA0-300_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1000_MA0-800_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1000_MA0-700_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1000_MA0-600_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1000_MA0-500_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1000_MA0-400_13TeV-madgraph
ZprimeToA0hToA0chichihbb_2HDM_MZp-1000_MA0-300_13TeV-madgraph
TTbarDMJets_scalar_Mchi-10_Mphi-100:(TTbarDMJets_scalar_Mchi-10_Mphi-100,MC,1),
TTbarDMJets_scalar_Mchi-10_Mphi-10:(TTbarDMJets_scalar_Mchi-10_Mphi-10,MC,1),
TTbarDMJets_scalar_Mchi-10_Mphi-15:(TTbarDMJets_scalar_Mchi-10_Mphi-15,MC,1),
TTbarDMJets_scalar_Mchi-10_Mphi-50:(TTbarDMJets_scalar_Mchi-10_Mphi-50,MC,1),
TTbarDMJets_scalar_Mchi-1_Mphi-10000:(TTbarDMJets_scalar_Mchi-1_Mphi-10000,MC,1),
TTbarDMJets_scalar_Mchi-1_Mphi-100:(TTbarDMJets_scalar_Mchi-1_Mphi-100,MC,1),
TTbarDMJets_scalar_Mchi-1_Mphi-10:(TTbarDMJets_scalar_Mchi-1_Mphi-10,MC,1),
TTbarDMJets_scalar_Mchi-1_Mphi-200:(TTbarDMJets_scalar_Mchi-1_Mphi-200,MC,1),
TTbarDMJets_scalar_Mchi-1_Mphi-20:(TTbarDMJets_scalar_Mchi-1_Mphi-20,MC,1),
TTbarDMJets_scalar_Mchi-1_Mphi-300:(TTbarDMJets_scalar_Mchi-1_Mphi-300,MC,1),
TTbarDMJets_scalar_Mchi-1_Mphi-500:(TTbarDMJets_scalar_Mchi-1_Mphi-500,MC,1),
TTbarDMJets_scalar_Mchi-1_Mphi-50:(TTbarDMJets_scalar_Mchi-1_Mphi-50,MC,1),
TTbarDMJets_scalar_Mchi-50_Mphi-10:(TTbarDMJets_scalar_Mchi-50_Mphi-10,MC,1),
TTbarDMJets_scalar_Mchi-50_Mphi-200:(TTbarDMJets_scalar_Mchi-50_Mphi-200,MC,1),
TTbarDMJets_scalar_Mchi-50_Mphi-300:(TTbarDMJets_scalar_Mchi-50_Mphi-300,MC,1),
TTbarDMJets_scalar_Mchi-50_Mphi-50:(TTbarDMJets_scalar_Mchi-50_Mphi-50,MC,1),
TTbarDMJets_scalar_Mchi-50_Mphi-95:(TTbarDMJets_scalar_Mchi-50_Mphi-95,MC,1),
BBbarDM_LO_MZprime-2500_Mhs-50_Mchi-200_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1000_Mhs-70_Mchi-150_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2000_Mhs-70_Mchi-250_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1000_Mhs-70_Mchi-250_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1000_Mhs-90_Mchi-300_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1500_Mhs-90_Mchi-400_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2500_Mhs-90_Mchi-250_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1000_Mhs-70_Mchi-50_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1500_Mhs-50_Mchi-50_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1000_Mhs-70_Mchi-400_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-500_Mhs-90_Mchi-50_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2000_Mhs-70_Mchi-150_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1500_Mhs-90_Mchi-250_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
DiJetsDM_LO_MZprime-1000_Mhs-50_Mchi-10_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
DiJetsDM_LO_MZprime-50_Mhs-150_Mchi-10_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-500_Mhs-90_Mchi-150_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1500_Mhs-70_Mchi-250_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1000_Mhs-50_Mchi-100_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2500_Mhs-90_Mchi-300_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2000_Mhs-50_Mchi-100_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1500_Mhs-90_Mchi-50_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2000_Mhs-70_Mchi-300_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1500_Mhs-50_Mchi-150_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
DiJetsDM_LO_MZprime-500_Mhs-50_Mchi-10_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
DiJetsDM_LO_MZprime-2000_Mhs-150_Mchi-10_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2000_Mhs-90_Mchi-400_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2500_Mhs-90_Mchi-400_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2500_Mhs-90_Mchi-150_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1500_Mhs-70_Mchi-300_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2500_Mhs-50_Mchi-300_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1500_Mhs-90_Mchi-300_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-500_Mhs-50_Mchi-50_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1500_Mhs-50_Mchi-250_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1000_Mhs-90_Mchi-150_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1500_Mhs-50_Mchi-400_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2000_Mhs-90_Mchi-150_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2500_Mhs-50_Mchi-150_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1000_Mhs-90_Mchi-400_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2500_Mhs-70_Mchi-150_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2500_Mhs-70_Mchi-200_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2000_Mhs-90_Mchi-250_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2000_Mhs-70_Mchi-400_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2000_Mhs-50_Mchi-400_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2000_Mhs-70_Mchi-200_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1000_Mhs-70_Mchi-100_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1000_Mhs-50_Mchi-200_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
DiJetsDM_LO_MZprime-300_Mhs-150_Mchi-10_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1500_Mhs-50_Mchi-300_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2500_Mhs-50_Mchi-250_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-500_Mhs-50_Mchi-150_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2000_Mhs-50_Mchi-250_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1500_Mhs-70_Mchi-200_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2000_Mhs-90_Mchi-300_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-500_Mhs-70_Mchi-150_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
DiJetsDM_LO_MZprime-300_Mhs-50_Mchi-10_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
DiJetsDM_LO_MZprime-100_Mhs-50_Mchi-10_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-500_Mhs-90_Mchi-200_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
DiJetsDM_LO_MZprime-2000_Mhs-50_Mchi-10_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-500_Mhs-70_Mchi-200_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
DiJetsDM_LO_MZprime-3000_Mhs-50_Mchi-10_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2000_Mhs-50_Mchi-50_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1500_Mhs-90_Mchi-200_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-500_Mhs-70_Mchi-100_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-500_Mhs-90_Mchi-100_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1500_Mhs-70_Mchi-100_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
DiJetsDM_LO_MZprime-500_Mhs-150_Mchi-10_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2500_Mhs-90_Mchi-100_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2500_Mhs-70_Mchi-300_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1000_Mhs-90_Mchi-50_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2000_Mhs-50_Mchi-150_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2000_Mhs-50_Mchi-300_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
DiJetsDM_LO_MZprime-1500_Mhs-50_Mchi-10_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1000_Mhs-90_Mchi-100_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2500_Mhs-70_Mchi-100_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1000_Mhs-70_Mchi-200_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1500_Mhs-50_Mchi-200_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2500_Mhs-70_Mchi-250_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2000_Mhs-90_Mchi-50_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2500_Mhs-50_Mchi-400_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
DiJetsDM_LO_MZprime-100_Mhs-150_Mchi-10_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
DiJetsDM_LO_MZprime-2500_Mhs-150_Mchi-10_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2000_Mhs-70_Mchi-100_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2000_Mhs-90_Mchi-200_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2500_Mhs-90_Mchi-200_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2500_Mhs-50_Mchi-100_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-500_Mhs-50_Mchi-100_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
DiJetsDM_LO_MZprime-50_Mhs-50_Mchi-10_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
DiJetsDM_LO_MZprime-1500_Mhs-150_Mchi-10_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2500_Mhs-90_Mchi-50_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1500_Mhs-70_Mchi-50_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-500_Mhs-70_Mchi-50_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2500_Mhs-70_Mchi-400_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2000_Mhs-70_Mchi-50_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1000_Mhs-90_Mchi-200_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1500_Mhs-90_Mchi-150_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1000_Mhs-50_Mchi-150_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1000_Mhs-50_Mchi-300_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2500_Mhs-70_Mchi-50_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1000_Mhs-50_Mchi-250_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2000_Mhs-50_Mchi-200_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
DiJetsDM_LO_MZprime-2500_Mhs-50_Mchi-10_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1500_Mhs-90_Mchi-100_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2500_Mhs-50_Mchi-50_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
DiJetsDM_LO_MZprime-1000_Mhs-150_Mchi-10_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1500_Mhs-70_Mchi-400_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1000_Mhs-70_Mchi-300_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1500_Mhs-50_Mchi-100_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-500_Mhs-50_Mchi-200_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1500_Mhs-70_Mchi-150_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
DiJetsDM_LO_MZprime-3000_Mhs-150_Mchi-10_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1000_Mhs-50_Mchi-50_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-1000_Mhs-90_Mchi-250_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-2000_Mhs-90_Mchi-100_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-3000_Mhs-90_Mchi-100_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-3000_Mhs-90_Mchi-150_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-3000_Mhs-90_Mchi-200_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-3000_Mhs-90_Mchi-250_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-3000_Mhs-90_Mchi-300_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-3000_Mhs-90_Mchi-400_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-3000_Mhs-90_Mchi-50_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-3000_Mhs-70_Mchi-100_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-3000_Mhs-70_Mchi-150_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-3000_Mhs-70_Mchi-200_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-3000_Mhs-70_Mchi-250_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-3000_Mhs-70_Mchi-300_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-3000_Mhs-70_Mchi-400_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-3000_Mhs-70_Mchi-50_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-495_Mhs-50_Mchi-250_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-495_Mhs-70_Mchi-250_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
BBbarDM_LO_MZprime-495_Mhs-90_Mchi-250_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
DiJetsDM_LO_MZprime-50_Mhs-150_Mchi-10_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
DiJetsDM_LO_MZprime-50_Mhs-50_Mchi-10_gSM-0p25_gDM-1p0_th_0p01_13TeV-madgraph
Vector_MonoTop_Leptonic_NLO_Mphi_300_Mchi_1_gSM_0p25_gDM_1p0_13TeV_madgraph
Vector_MonoTop_Leptonic_NLO_Mphi_300_Mchi_200_gSM_0p25_gDM_1p0_13TeV_madgraph
Vector_MonoTop_Leptonic_NLO_Mphi_300_Mchi_500_gSM_0p25_gDM_1p0_13TeV_madgraph
Vector_MonoTop_Leptonic_NLO_Mphi_300_Mchi_1000_gSM_0p25_gDM_1p0_13TeV_madgraph
Vector_MonoTop_Leptonic_NLO_Mphi_1000_Mchi_1000_gSM_0p25_gDM_1p0_13TeV_madgraph
Vector_MonoTop_Leptonic_NLO_Mphi_1000_Mchi_1_gSM_0p25_gDM_1p0_13TeV_madgraph
Vector_MonoTop_Leptonic_NLO_Mphi_1000_Mchi_200_gSM_0p25_gDM_1p0_13TeV_madgraph
Vector_MonoTop_Leptonic_NLO_Mphi_1000_Mchi_490_gSM_0p25_gDM_1p0_13TeV_madgraph
Vector_MonoTop_Leptonic_NLO_Mphi_1500_Mchi_1000_gSM_0p25_gDM_1p0_13TeV_madgraph
Vector_MonoTop_Leptonic_NLO_Mphi_1500_Mchi_1_gSM_0p25_gDM_1p0_13TeV_madgraph
Vector_MonoTop_Leptonic_NLO_Mphi_1500_Mchi_200_gSM_0p25_gDM_1p0_13TeV_madgraph
Vector_MonoTop_Leptonic_NLO_Mphi_1500_Mchi_500_gSM_0p25_gDM_1p0_13TeV_madgraph
Vector_MonoTop_Leptonic_NLO_Mphi_1750_Mchi_1000_gSM_0p25_gDM_1p0_13TeV_madgraph
Vector_MonoTop_Leptonic_NLO_Mphi_1750_Mchi_1_gSM_0p25_gDM_1p0_13TeV_madgraph
Vector_MonoTop_Leptonic_NLO_Mphi_1750_Mchi_200_gSM_0p25_gDM_1p0_13TeV_madgraph
Vector_MonoTop_Leptonic_NLO_Mphi_1750_Mchi_500_gSM_0p25_gDM_1p0_13TeV_madgraph
Vector_MonoTop_Leptonic_NLO_Mphi_2000_Mchi_1_gSM_0p25_gDM_1p0_13TeV_madgraph
Vector_MonoTop_Leptonic_NLO_Mphi_2000_Mchi_200_gSM_0p25_gDM_1p0_13TeV_madgraph
Vector_MonoTop_Leptonic_NLO_Mphi_2000_Mchi_500_gSM_0p25_gDM_1p0_13TeV_madgraph
Vector_MonoTop_Leptonic_NLO_Mphi_2000_Mchi_990_gSM_0p25_gDM_1p0_13TeV_madgraph
Vector_MonoTop_Leptonic_NLO_Mphi_2250_Mchi_1_gSM_0p25_gDM_1p0_13TeV_madgraph
Vector_MonoTop_Leptonic_NLO_Mphi_2500_Mchi_1000_gSM_0p25_gDM_1p0_13TeV_madgraph
Vector_MonoTop_Leptonic_NLO_Mphi_2500_Mchi_1_gSM_0p25_gDM_1p0_13TeV_madgraph
Vector_MonoTop_Leptonic_NLO_Mphi_2500_Mchi_200_gSM_0p25_gDM_1p0_13TeV_madgraph
Vector_MonoTop_Leptonic_NLO_Mphi_2500_Mchi_500_gSM_0p25_gDM_1p0_13TeV_madgraph
Vector_MonoTop_Leptonic_NLO_Mphi_500_Mchi_1000_gSM_0p25_gDM_1p0_13TeV_madgraph
Vector_MonoTop_Leptonic_NLO_Mphi_500_Mchi_1_gSM_0p25_gDM_1p0_13TeV_madgraph
Vector_MonoTop_Leptonic_NLO_Mphi_500_Mchi_500_gSM_0p25_gDM_1p0_13TeV_madgraph
Vector_MonoTop_Leptonic_NLO_Mphi_750_Mchi_1000_gSM_0p25_gDM_1p0_13TeV_madgraph
Vector_MonoTop_Leptonic_NLO_Mphi_750_Mchi_1_gSM_0p25_gDM_1p0_13TeV_madgraph
Vector_MonoTop_Leptonic_NLO_Mphi_750_Mchi_200_gSM_0p25_gDM_1p0_13TeV_madgraph
Vector_MonoTop_Leptonic_NLO_Mphi_750_Mchi_500_gSM_0p25_gDM_1p0_13TeV_madgraph
""".split()
    


bg_datasets = [ds.strip() for ds in bg_datasets if ds.strip()]
signal_samples = [ds.strip() for ds in signal_samples if ds.strip()]
def category(ds):
        
         
      
        if "DYJetsToLL_M-50_TuneCUETP8M1" in ds:  return "ZJets_nlo"
        if "DYJetsToNuNu_TuneCUETP8M1" in ds:  return "ZtoNuNu_nlo"
        if "WJetsToLNu_TuneCUETP8M1" in ds:  return "WJets_nlo"
        if "ZJetsToNuNu_HT-100To200" in ds:  return "ZtoNuNu_ht100to200"
        if "ZJetsToNuNu_HT-200To400" in ds:  return "ZtoNuNu_ht200to400"
        if "ZJetsToNuNu_HT-400To600" in ds:  return "ZtoNuNu_ht400to600"
        if "ZJetsToNuNu_HT-600To800" in ds:  return "ZtoNuNu_ht600to800"
        if "ZJetsToNuNu_HT-800To1200" in ds:  return "ZtoNuNu_ht800to1200"
        if "ZJetsToNuNu_HT-1200To2500" in ds:  return "ZtoNuNu_ht1200to2500"
        if " ZJetsToNuNu_HT-2500ToInf" in ds:  return "ZtoNuNu_ht2500to inf"
        if " ZJetsToNuNu_HT-600ToInf" in ds:  return "ZtoNuNu_ht600to inf"     
        if "DYJetsToLL_M-50_HT-70to100" in ds:  return "ZJets_ht50to100"
        if "DYJetsToLL_M-50_HT-100to200" in ds:  return "ZJets_ht100to200"
        if "DYJetsToLL_M-50_HT-200to400" in ds:  return "ZJets_ht200to400"
        if "DYJetsToLL_M-50_HT-400to600" in ds:  return "ZJets_ht400to600"
        if "DYJetsToLL_M-50_HT-600toInf" in ds:  return "ZJets_ht600to inf"
        if "DYJetsToLL_M-50_HT-1200to2500" in ds:  return "ZJets_ht1200to2500"
        if "DYJetsToLL_M-50_HT-2500toInf" in ds:  return "ZJets_ht2500to inf"
        if "DYJetsToLL_M-50_HT-600to800" in ds:  return "ZJets_ht600to800"
        if "DYJetsToLL_M-50_HT-800to1200" in ds:  return "ZJets_ht800to1200"
        if "WJetsToLNu_HT-70To100" in ds:  return "WJets_ht70to100"
        if "WJetsToLNu_HT-100To200" in ds:  return "WJets_ht100to200"
        if "WJetsToLNu_HT-200To400" in ds:  return "WJets_ht200to400"
        if "WJetsToLNu_HT-400To600" in ds:  return "WJets_ht400to600"
        if "WJetsToLNu_HT-600To800" in ds:  return "WJets_ht600to800"
        if "WJetsToLNu_HT-800To1200" in ds:  return "WJets_ht800to1200"
        if "WJetsToLNu_HT-1200To2500" in ds:  return "WJets_ht1200to2500"
        if "WJetsToLNu_HT-2500ToInf" in ds:  return "WJets_ht2500to inf"
        if "WJetsToLNu_HT-600ToInf" in ds:  return "WJets_ht600to inf"
        if "WJetsToLNu_Pt-100To250" in ds:  return "WJets_pt100to250"
        if "WJetsToLNu_Pt-250To400" in ds:  return "WJets_pt250to400"
        if "WJetsToLNu_Pt-400To600" in ds:  return "WJets_pt400to600"
        if "WJetsToLNu_Pt-600ToInf" in ds:  return "WJets_pt600to inf"
        if "DYJetsToLL_Pt-50To100" in ds:  return "ZJets_pt50to100"
        if "DYJetsToLL_Pt-100To250" in ds:  return "ZJets_pt100to250"
        if "DYJetsToLL_Pt-250To400" in ds:  return "ZJets_pt250to400"
        if "DYJetsToLL_Pt-400To650" in ds:  return "ZJets_pt400to650"
        if "DYJetsToLL_Pt-650ToInf" in ds:  return "ZJets_pt650to inf"
        if "DYJetsToNuNu_PtZ-100To250" in ds:  return "ZtoNuNu_pt100to250"
        if "DYJetsToNuNu_PtZ-250To400" in ds:  return "ZtoNuNu_pt250to400"
        if "DYJetsToNuNu_PtZ-400To650" in ds:  return "ZtoNuNu_pt400to650"
        if "DYJetsToNuNu_PtZ-650ToInf" in ds:  return "ZtoNuNu_pt650to inf"
        if "GJets_HT-40To100" in ds:  return "GJets_ht40to100"
        if "GJets_HT-100To200" in ds:  return "GJets_ht100to200"
        if "GJets_HT-200To400" in ds:  return "GJets_ht200to400"
        if "GJets_HT-400To600" in ds:  return "GJets_ht400to600"
        if "GJets_HT-600ToInf" in ds:  return "GJets_ht600to inf"
        if "QCD_HT50to100" in ds:  return "QCD_ht50to100"
        if "QCD_HT100to200" in ds:  return "QCD_ht100to200"
        if "QCD_HT200to300" in ds:  return "QCD_ht200to300"
        if "QCD_HT300to500" in ds:  return "QCD_ht300to500"
        if "QCD_HT500to700" in ds:  return "QCD_ht500to700"
        if "QCD_HT700to1000" in ds: return "QCD_ht700to1000"
        if "QCD_HT1000to1500" in ds: return "QCD_ht1000to1500"
        if "QCD_HT1500to2000" in ds: return "QCD_ht1500to2000"
        if "QCD_HT2000toInf" in ds:  return "QCD_ht2000to inf"
        if "WW_TuneCUETP8M1" in ds:  return "Diboson_ww"
        if "WZ_TuneCUETP8M1" in ds:  return "Diboson_wz"
        if "ZZ_TuneCUETP8M1" in ds:  return "Diboson_zz"
        if "WWTo2L2Nu" in ds:  return "WWTo2L2Nu"
        if "WWTo4Q" in ds:  return "WWTo4Q"
        if "WWToLNuQQ" in ds:  return "WWToLNuQQ"
        if "WZTo1L1Nu2Q"  in ds:  return "WZTo1L1Nu2Q"
        if "WZTo1L3Nu"  in ds:  return "WZTo1L3Nu"
        if "WZTo2L2Q" in ds:  return "WZTo2L2Q"
        if "WZTo3LNu" in ds:  return "WZTo3LNu"
        if "ZZTo2L2Nu" in ds:  return "ZZTo2L2Nu"
        if "ZZTo2L2Q"  in ds:  return "ZZTo2L2Q"
        if "ZZTo4L" in ds:  return "ZZTo4L"
        if  "ZH_HToBB_ZToNuNu" in ds:  return "ZnunuH"
        if "ZH_HToBB_ZToNuNu" in ds:  return "ZnunuH"
        if  "ZH_HToBB_ZToLL" in ds:  return "ZllH"
        if"ZH_HToBB_ZToQQ" in ds:  return "ZqqH"
        if"ggZH_HToBB_ZToNuNu" in ds:  return "ggZnunuH"
        if"ggZH_HToBB_ZToLL" in ds:  return "ggZllH"
        if"WminusH_HToBB_WToLNu" in ds:  return "WmH"
        if "WplusH_HToBB_WToLNu" in ds:  return "WpH"
        if"WplusH_HToBB_WToQQ" in ds:  return "WpH_qq"
        if"WminusH_HToBB_WToQQ" in ds:  return "WmH_qq"
        if "WWTo4Q_4f" in ds:  return "WW_4q"
        if "WJetsToQQ_HT180" in ds:  return "WJets_qq"
        if"ZZTo4Q" in ds:  return "ZZ_4q"
        if "DYJetsToQQ_HT180" in ds:  return "ZJets_qq"
        if "ttHTobb" in ds:  return "ttH"
        if "ttHTobb" in ds:  return "ttHtranche3"
        if "GluGluHToBB" in ds:  return "ggH"
        if "VBFHToBB_M125_13TeV_amcatnlo" in ds:  return "VBFH"
        if "VBFHToBB_M-125_13TeV_powheg" in ds:  return "VBFH_powheg"
        if "GluGluToCont inToZZTo2e2nu" in ds:  return "ggZZTo2e2nu"
        if "GluGluToCont inToZZTo2mu2nu" in ds:  return "ggZZTo2mu2tau"
        if "GluGluToContinToZZTo2e2mu" in ds:  return "ggZZTo2e2mu"
        if "GluGluToContinToZZTo2e2tau" in ds:  return "ggZZTo2e2tau"
        if "GluGluToContinToZZTo2mu2tau" in ds:  return "ggZZTo2mu2tau"
        if "GluGluToContinToZZTo4e" in ds:  return "ggZZTo4e"
        if "GluGluToContinToZZTo4mu" in ds:  return "ggZZTo4mu"
        if "GluGluToContinToZZTo4tau" in ds:  return "ggZZTo4tau"
        if "ST_tW_top_5f" in ds:  return "SingleTop_tW"
        if "ST_tW_antitop_5f" in ds:  return "SingleTop_tbarW"
        if "ST_t-channel_antitop_4f_leptonDecays" in ds: return "SingleTop_tTbar_lep"
        if "ST_t-channel_top_4f_leptonDecays" in ds and  not ("TuneCUETP8M1" in ds): return "SingleTop_tT_lep"
        if "ST_tW_top_5f" in ds: return "SingleTop_tW"
        if "ST_tW_antitop_5f" in ds: return  "SingleTop_tbar"
        if "ST_t-channel_antitop_4f"  in ds: return "SingleTop_tTbar"
        if "ST_t-channel_top_4f_inclusiveDecays" in ds: return "SingleTop_tT"
        if "ST_s-channel_4f_leptonDecays" in ds: return "SingleTop_s_lep"
        if "ST_t-channel_5f_leptonDecays" in ds: return "SingleTop_t_lep"
        if "ST_t-channel_top_4f_leptonDecays" in ds: return "SingleTop_tchannel"
        if "TTJets_TuneCUETP8M1"    in ds: return  "TTbar_MLM"
        if "TTJets_DiLept_TuneCUETP8M1"   in ds: return  "TTbar_2L"
        if "TTJets_SingleLeptFromT_TuneCUETP8M1"  in ds: return "TTbar_1LT"
        if "TTJets_SingleLeptFromTbar_TuneCUETP8M1" in ds: return "TTbar_1LTbar"
        if "TTJets_SingleLeptFromT_TuneCUETP8M2T4" in ds: return "TTbar_FXFX_1LT"
        if "TTJets_SingleLeptFromTbar_TuneCUETP8M2T4" in ds: return "TTbar_FXFX_1LTbar"
        if "TT_TuneEE5C_13TeV-powheg-herwigpp" in ds: return "TTbar_Herwig"
        if "TTJets_TuneCUETP8M2T4_13TeV-amcatnloFXFX-pythia8"   in ds: return     "TTbar_FXFX"
        if "TT_TuneCUETP8M2T4_13TeV-powheg-pythia8"  in ds: return  "TTbar_Powheg"
        if "TT_TuneCUETP8M2T4_13TeV-powheg-isrdown" in ds: return  "TTbar_PowhegISRDown"
        if "TT_TuneCUETP8M2T4_13TeV-powheg-isrup"   in ds: return  "TTbar_PowhegISRUp"
        if "TT_TuneCUETP8M2T4down"  in ds: return   "TTbar_PowhegTuneDown"
        if "TT_TuneCUETP8M2T4up"    in ds: return   "TTbar_PowhegTuneUp"
        if "TTTo2L2Nu_TuneCUETP8M2_ttHtranche3" in ds: return "TTTo2L2Nu"
        if "EWKZ2Jets_ZToNuNu" in ds:  return "ZtoNuNu_EWK"
        if "EWKZ2Jets_ZToLL" in ds:  return "ZJets_EWK"
        if "EWKWPlus2Jets_WToLNu" in ds:  return "WJets_EWKWPlus"
        if "EWKWMinus2Jets_WToLNu" in ds:  return "WJets_EWKWMinus" 
        if "tZq_ll_4f" in ds:  return "SingleTop_tZll"
        if "tZq_nunu_4f" in ds:  return "SingleTop_tZnunu"
        if "TTGJets_TuneCUETP8M1" in ds:  return "TTbar_GJets"
        if "TGJets_TuneCUETP8M1" in ds:  return "SingleTop_tG"

        
def luminosity(ds):
        if "thq" in ds: return 0.7927
        if "thw" in ds: return 0.147
        if "DYJetsToLL_M-50_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8" in ds: return 6025.2
        if "DYJetsToNuNu_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8" in ds: return 11433.
        if "WJetsToLNu_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8" in ds: return 61527.
        if "ZJetsToNuNu_HT-100To200_13TeV-madgraph" in ds: return 280.5
        if "ZJetsToNuNu_HT-200To400_13TeV-madgraph" in ds: return 77.7
        if "ZJetsToNuNu_HT-400To600_13TeV-madgraph" in ds: return 10.71
        if "ZJetsToNuNu_HT-600To800_13TeV-madgraph" in ds: return 2.562
        if "ZJetsToNuNu_HT-800To1200_13TeV-madgraph" in ds: return 1.183
        if "ZJetsToNuNu_HT-1200To2500_13TeV-madgraph" in ds: return 0.286
        if "ZJetsToNuNu_HT-2500ToInf_13TeV-madgraph" in ds: return 0.006945
        if "ZJetsToNuNu_HT-600ToInf_13TeV-madgraph" in ds: return 4.098
        if "DYJetsToLL_M-50_HT-70to100_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 175.3
        if "DYJetsToLL_M-50_HT-100to200_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 148.
        if "DYJetsToLL_M-50_HT-200to400_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 40.94
        if "DYJetsToLL_M-50_HT-400to600_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 5.497
        if "DYJetsToLL_M-50_HT-600toInf_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 2.193
        if "DYJetsToLL_M-50_HT-1200to2500_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 0.1514
        if "DYJetsToLL_M-50_HT-2500toInf_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 0.003565
        if "DYJetsToLL_M-50_HT-600to800_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 1.367
        if "DYJetsToLL_M-50_HT-800to1200_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 0.6304
        if "WJetsToLNu_HT-70To100_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 1319
        if "WJetsToLNu_HT-100To200_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 1343
        if "WJetsToLNu_HT-200To400_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 359.6
        if "WJetsToLNu_HT-400To600_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 48.85
        if "WJetsToLNu_HT-600To800_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 12.05
        if "WJetsToLNu_HT-800To1200_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 5.501
        if "WJetsToLNu_HT-1200To2500_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 1.329
        if "WJetsToLNu_HT-2500ToInf_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 0.03216
        if "WJetsToLNu_HT-600ToInf_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 18.91
        if "WJetsToLNu_Pt-100To250_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8" in ds: return 677.82
        if "WJetsToLNu_Pt-250To400_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8" in ds: return 24.083
        if "WJetsToLNu_Pt-400To600_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8" in ds: return 3.0563
        if "WJetsToLNu_Pt-600ToInf_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8" in ds: return 0.4602
        if "DYJetsToLL_Pt-50To100_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8" in ds: return 374.6800
        if "DYJetsToLL_Pt-100To250_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8" in ds: return 86.5200
        if "DYJetsToLL_Pt-250To400_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8" in ds: return 3.3247
        if "DYJetsToLL_Pt-400To650_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8" in ds: return 0.4491
        if "DYJetsToLL_Pt-650ToInf_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8" in ds: return 0.0422
        if "DYJetsToNuNu_PtZ-100To250_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8" in ds: return 3*54.8229
        if "DYJetsToNuNu_PtZ-250To400_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8" in ds: return 3*2.0705
        if "DYJetsToNuNu_PtZ-400To650_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8" in ds: return 3*0.2779
        if "DYJetsToNuNu_PtZ-650ToInf_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8" in ds: return 3*0.0261                             
        if "GJets_HT-40To100_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 23080.0
        if "GJets_HT-100To200_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 9235
        if "GJets_HT-200To400_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 2298
        if "GJets_HT-400To600_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 277.6
        if "GJets_HT-600ToInf_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 93.47  
        if "QCD_HT50to100_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 278700000
        if "QCD_HT100to200_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 27990000
        if "QCD_HT200to300_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 1735000
        if "QCD_HT300to500_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 366800
        if "QCD_HT500to700_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 29370
        if "QCD_HT700to1000_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 6524
        if "QCD_HT1000to1500_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 1064
        if "QCD_HT1500to2000_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 121.5
        if "QCD_HT2000toInf_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 25.42  
        if "ST_t-channel_antitop_4f_leptonDecays_13TeV-powheg-pythia8" in ds: return 26.22
        if "ST_t-channel_top_4f_leptonDecays_13TeV-powheg-pythia8" in ds: return 44.07
        if "ST_tW_top_5f_inclusiveDecays_13TeV-powheg-pythia8_TuneCUETP8M1" in ds: return 35.85
        if "ST_tW_antitop_5f_inclusiveDecays_13TeV-powheg-pythia8_TuneCUETP8M1" in ds: return 35.85
        if "ST_t-channel_antitop_4f_inclusiveDecays_13TeV-powhegV2-madspin-pythia8_TuneCUETP8M1" in ds: return 80.95
        if "ST_t-channel_top_4f_inclusiveDecays_13TeV-powhegV2-madspin-pythia8_TuneCUETP8M1" in ds: return 136.02
        if "ST_s-channel_4f_leptonDecays_13TeV-amcatnlo-pythia8_TuneCUETP8M1" in ds: return 10.11
        if "ST_t-channel_5f_leptonDecays_13TeV-amcatnlo-pythia8_TuneCUETP8M1" in ds: return 216.99
        if "ST_t-channel_top_4f_leptonDecays_13TeV-powheg-pythia8_TuneCUETP8M1" in ds: return 44.3
        if "tZq_ll_4f_13TeV-amcatnlo-pythia8" in ds: return 0.0758
        if "tZq_nunu_4f_13TeV-amcatnlo-pythia8_TuneCUETP8M1" in ds: return 0.1379
        if "TTGJets_TuneCUETP8M1_13TeV-amcatnloFXFX-madspin-pythia8" in ds: return 3.786
        if "TGJets_TuneCUETP8M1_13TeV_amcatnlo_madspin_pythia8" in ds: return 2.967       
        if "EWKZ2Jets_ZToNuNu_13TeV-madgraph-pythia8" in ds: return 10.04
        if "EWKZ2Jets_ZToLL_M-50_13TeV-madgraph-pythia8" in ds: return 3.99
        if "EWKWPlus2Jets_WToLNu_M-50_13TeV-madgraph-pythia8" in ds: return 25.81
        if "EWKWMinus2Jets_WToLNu_M-50_13TeV-madgraph-pythia8" in ds: return 20.35       
        if "WW_TuneCUETP8M1_13TeV-pythia8" in ds: return 118.7
        if "WZ_TuneCUETP8M1_13TeV-pythia8" in ds: return 47.13
        if "ZZ_TuneCUETP8M1_13TeV-pythia8" in ds: return 16.523      
        if "ZH_HToBB_ZToNuNu_M125_13TeV_powheg_pythia8" in ds: return 0.08912
        if "ZH_HToBB_ZToNuNu_M125_13TeV_powheg_pythia8" in ds: return 0.884*0.582*0.20
        if "ZH_HToBB_ZToLL_M125_13TeV_powheg_pythia8" in ds: return 0.04865
        if "ZH_HToBB_ZToQQ_M125_13TeV_powheg_pythia8" in ds: return 0.607*0.582
        if "ggZH_HToBB_ZToNuNu_M125_13TeV_powheg_pythia8" in ds: return 0.014366
        if "ggZH_HToBB_ZToLL_M125_13TeV_powheg_pythia8" in ds: return 0.007842
        if "WminusH_HToBB_WToLNu_M125_13TeV_powheg_pythia8" in ds: return 0.100
        if "WplusH_HToBB_WToLNu_M125_13TeV_powheg_pythia8" in ds: return 0.159
        if "WplusH_HToBB_WToQQ_M125_13TeV_powheg_pythia8" in ds: return 0.84*(1-3*0.108)*0.582
        if "WminusH_HToBB_WToQQ_M125_13TeV_powheg_pythia8" in ds: return 0.533*(1-3*0.108)*0.582
        if "WWTo4Q_4f_13TeV_amcatnloFXFX_madspin_pythia8" in ds: return 51.723
        if "WJetsToQQ_HT180_13TeV-madgraphMLM-pythia8" in ds: return 2788
        if "ZZTo4Q_13TeV_amcatnloFXFX_madspin_pythia8" in ds: return 6.842
        if "DYJetsToQQ_HT180_13TeV-madgraphMLM-pythia8" in ds: return 1187
        if "ttHTobb_M125_13TeV_powheg_pythia8" in ds: return 0.506*0.5824
        if "ttHTobb_M125_TuneCUETP8M2_ttHtranche3_13TeV-powheg-pythia8" in ds: return 0.506*0.5824
        if "GluGluHToBB_M125_13TeV_powheg_pythia8" in ds: return 48.48*0.5824
        if "VBFHToBB_M125_13TeV_amcatnlo_pythia8" in ds: return 3.782*0.5824
        if "VBFHToBB_M-125_13TeV_powheg_pythia8_weightfix" in ds: return 3.782*0.5824     
        if "TTJets_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 831.76
        if "TTJets_DiLept_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 831.76*(1-0.68)*(1-0.68)
        if "TTJets_SingleLeptFromT_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 831.76*0.68*(1-0.68)
        if "TTJets_SingleLeptFromTbar_TuneCUETP8M1_13TeV-madgraphMLM-pythia8" in ds: return 831.76*0.68*(1-0.68)
        if "TTJets_SingleLeptFromT_TuneCUETP8M2T4_13TeV-amcatnloFXFX-pythia8" in ds: return 831.76*0.68*(1-0.68)
        if "TTJets_SingleLeptFromTbar_TuneCUETP8M2T4_13TeV-amcatnloFXFX-pythia8" in ds: return 831.76*0.68*(1-0.68)
        if "TT_TuneEE5C_13TeV-powheg-herwigpp" in ds: return 831.76
        if "TTJets_TuneCUETP8M2T4_13TeV-amcatnloFXFX-pythia8" in ds: return 831.76
        if "TT_TuneCUETP8M2T4_13TeV-powheg-pythia8" in ds: return 831.76
        if "TT_TuneCUETP8M2T4_13TeV-powheg-isrdown-pythia8" in ds: return 831.76
        if "TT_TuneCUETP8M2T4_13TeV-powheg-isrup-pythia8" in ds: return 831.76
        if "TT_TuneCUETP8M2T4down_13TeV-powheg-pythia8" in ds: return 831.76
        if "TT_TuneCUETP8M2T4up_13TeV-powheg-pythia8" in ds: return 831.76
        if "TTTo2L2Nu_TuneCUETP8M2_ttHtranche3_13TeV-powheg-pythia8" in ds: return 88.288           
        if "WWTo2L2Nu_13TeV-powheg" in ds: return (118.7-3.974)*0.1086*0.1086*9
        if "WWTo4Q_13TeV-powheg" in ds: return 51.723
        if "WWToLNuQQ_13TeV-powheg" in ds: return 49.997
        if "WZTo1L1Nu2Q_13TeV_amcatnloFXFX_madspin_pythia8" in ds: return 10.71
        if "WZTo1L3Nu_13TeV_amcatnloFXFX_madspin_pythia8" in ds: return 3.033
        if "WZTo2L2Q_13TeV_amcatnloFXFX_madspin_pythia8" in ds: return 5.595
        if "WZTo3LNu_TuneCUETP8M1_13TeV-powheg-pythia8" in ds: return 4.430
        if "ZZTo2L2Nu_13TeV_powheg_pythia8" in ds: return 0.5644
        if "ZZTo2L2Q_13TeV_amcatnloFXFX_madspin_pythia8" in ds: return 3.22
        if "ZZTo4L_13TeV_powheg_pythia8" in ds: return 1.212
        if "GluGluToContinToZZTo2e2nu_13TeV_MCFM701_pythia8" in ds: return 0.003956
        if "GluGluToContinToZZTo2mu2nu_13TeV_MCFM701_pythia8" in ds: return 0.003956
        if "GluGluToContinToZZTo2e2mu_13TeV_MCFM701_pythia8" in ds: return 0.0073462
        if "GluGluToContinToZZTo2e2tau_13TeV_MCFM701_pythia8" in ds: return 0.0073462
        if "GluGluToContinToZZTo2mu2tau_13TeV_MCFM701_pythia8" in ds: return 0.0073462
        if "GluGluToContinToZZTo4e_13TeV_MCFM701_pythia8" in ds: return 0.0036478
        if "GluGluToContinToZZTo4mu_13TeV_MCFM701_pythia8" in ds: return 0.0036478
        if "GluGluToContinToZZTo4tau_13TeV_MCFM701_pythia8" in ds: return 0.0036478       
        if "GluGlu_HToInvisible_M125_13TeV_powheg_pythia8" in ds: return 48.6
        if "VBF_HToInvisible_M125_13TeV_powheg_pythia8" in ds: return 3.78
        if "VBF_HToInvisible_M110_13TeV_powheg_pythia8" in ds: return 4.34
        if "VBF_HToInvisible_M150_13TeV_powheg_pythia8" in ds: return 3.239
        if "VBF_HToInvisible_M200_13TeV_powheg_pythia8" in ds: return 2.282
        if "VBF_HToInvisible_M300_13TeV_powheg_pythia8" in ds: return 1.256
        if "VBF_HToInvisible_M500_13TeV_powheg_pythia8" in ds: return 0.4872
        if "VBF_HToInvisible_M600_13TeV_powheg_pythia8" in ds: return 0.3274
        if "VBF_HToInvisible_M800_13TeV_powheg_pythia8" in ds: return 0.1622
        if "VBF_HToInvisible_M1000_13TeV_powheg_pythia8" in ds: return 0.08732
        if "GluGlu_HToInvisible_M110_13TeV_powheg_pythia8" in ds: return 57.90
        if "GluGlu_HToInvisible_M150_13TeV_powheg_pythia8" in ds: return 31.29
        if "GluGlu_HToInvisible_M200_13TeV_powheg_pythia8" in ds: return 16.94
        if "GluGlu_HToInvisible_M300_13TeV_powheg_pythia8" in ds: return 6.590
        if "GluGlu_HToInvisible_M500_13TeV_powheg_pythia8" in ds: return 1.709
        if "GluGlu_HToInvisible_M600_13TeV_powheg_pythia8" in ds: return 1.001
        if "GluGlu_HToInvisible_M800_13TeV_powheg_pythia8" in ds: return 0.4015
        if "GluGlu_HToInvisible_M1000_13TeV_powheg_pythia8" in ds: return 0.1845
        if "thq" in ds: return 0.7927
        if "thw" in ds: return 0.147
        if "ST_tch_DM-scalar_LO-100_1-13_TeV" in ds: return 0.293*0.68
        if "ST_tch_DM-scalar_LO-300_1-13_TeV" in ds: return 0.03202*0.68
        if "ST_tch_DM-scalar_LO-500_1-13_TeV" in ds: return 0.004996*0.68
        if "ST_tch_DM-scalar_LO-1000_1-13_TeV" in ds: return 0.0003009*0.68
        if "TT_DM-scalar_LO-300_1-13_TeV" in ds: return 0.03045
        if "TT_DM-scalar_LO-500_1-13_TeV" in ds: return 0.004947
        if "TT_DM-scalar_LO-1000_1-13_TeV" in ds: return 0.000736
        if "MonoHbb_ZpBaryonic_MZp-10_MChi-1_13TeV-madgraph" in ds: return 1.4971719048020
        if "MonoHbb_ZpBaryonic_MZp-10_MChi-10_13TeV-madgraph" in ds: return 0.0068827725015
        if "MonoHbb_ZpBaryonic_MZp-10_MChi-50_13TeV-madgraph" in ds: return 0.0001182107976
        if "MonoHbb_ZpBaryonic_MZp-10_MChi-150_13TeV-madgraph" in ds: return 0.0000102513973
        if "MonoHbb_ZpBaryonic_MZp-10_MChi-500_13TeV-madgraph" in ds: return 0.0000000026006
        if "MonoHbb_ZpBaryonic_MZp-10_MChi-1000_13TeV-madgraph" in ds: return 0.0000000000160
        if "MonoHbb_ZpBaryonic_MZp-15_MChi-10_13TeV-madgraph" in ds: return 0.0241243264082
        if "MonoHbb_ZpBaryonic_MZp-20_MChi-1_13TeV-madgraph" in ds: return 1.5708303614837
        if "MonoHbb_ZpBaryonic_MZp-50_MChi-1_13TeV-madgraph" in ds: return 1.8787525219888
        if "MonoHbb_ZpBaryonic_MZp-50_MChi-10_13TeV-madgraph" in ds: return 1.8686472610676
        if "MonoHbb_ZpBaryonic_MZp-50_MChi-50_13TeV-madgraph" in ds: return 0.0041445173799
        if "MonoHbb_ZpBaryonic_MZp-95_MChi-50_13TeV-madgraph" in ds: return 0.1070881411920
        if "MonoHbb_ZpBaryonic_MZp-100_MChi-1_13TeV-madgraph" in ds: return 1.8350014490170
        if "MonoHbb_ZpBaryonic_MZp-100_MChi-10_13TeV-madgraph" in ds: return 1.8311374853540
        if "MonoHbb_ZpBaryonic_MZp-200_MChi-1_13TeV-madgraph" in ds: return 1.4723542212437
        if "MonoHbb_ZpBaryonic_MZp-200_MChi-50_13TeV-madgraph" in ds: return 1.2073355514877
        if "MonoHbb_ZpBaryonic_MZp-200_MChi-150_13TeV-madgraph" in ds: return 0.0018402642784
        if "MonoHbb_ZpBaryonic_MZp-295_MChi-150_13TeV-madgraph" in ds: return 0.0772562835636
        if "MonoHbb_ZpBaryonic_MZp-300_MChi-1_13TeV-madgraph" in ds: return 1.3081597528204
        if "MonoHbb_ZpBaryonic_MZp-300_MChi-50_13TeV-madgraph" in ds: return 1.1234969709678
        if "MonoHbb_ZpBaryonic_MZp-500_MChi-1_13TeV-madgraph" in ds: return 0.6310480800692
        if "MonoHbb_ZpBaryonic_MZp-500_MChi-150_13TeV-madgraph" in ds: return 0.3922022706199
        if "MonoHbb_ZpBaryonic_MZp-500_MChi-500_13TeV-madgraph" in ds: return 0.0000118690907
        if "MonoHbb_ZpBaryonic_MZp-995_MChi-500_13TeV-madgraph" in ds: return 0.0000000466117
        if "MonoHbb_ZpBaryonic_MZp-1000_MChi-1_13TeV-madgraph" in ds: return 0.1164211478301
        if "MonoHbb_ZpBaryonic_MZp-1000_MChi-150_13TeV-madgraph" in ds: return 0.1130981455808
        if "MonoHbb_ZpBaryonic_MZp-1000_MChi-1000_13TeV-madgraph" in ds: return 0.0000003182419
        if "MonoHbb_ZpBaryonic_MZp-1995_MChi-1000_13TeV-madgraph" in ds: return 0.0004210309298
        if "MonoHbb_ZpBaryonic_MZp-2000_MChi-1_13TeV-madgraph" in ds: return 0.0080403926209
        if "MonoHbb_ZpBaryonic_MZp-2000_MChi-500_13TeV-madgraph" in ds: return 0.0073116588302
        if "MonoHbb_ZpBaryonic_MZp-10000_MChi-1_13TeV-madgraph" in ds: return 0.0000000068756
        if "MonoHbb_ZpBaryonic_MZp-10000_MChi-10_13TeV-madgraph" in ds: return 0.0000000068844
        if "MonoHbb_ZpBaryonic_MZp-10000_MChi-50_13TeV-madgraph" in ds: return 0.0000000068310
        if "MonoHbb_ZpBaryonic_MZp-10000_MChi-150_13TeV-madgraph" in ds: return 0.0000000066844
        if "MonoHbb_ZpBaryonic_MZp-10000_MChi-500_13TeV-madgraph" in ds: return 0.0000000055185
        if "MonoHbb_ZpBaryonic_MZp-10000_MChi-1000_13TeV-madgraph" in ds: return 0.0000000038070
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-800_MA0-600_13TeV-madgraph" in ds: return 0.010101*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-800_MA0-500_13TeV-madgraph" in ds: return 0.0367*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-800_MA0-400_13TeV-madgraph" in ds: return 0.091124*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-800_MA0-300_13TeV-madgraph" in ds: return 0.27765*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-600_MA0-400_13TeV-madgraph" in ds: return 0.063916*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-600_MA0-300_13TeV-madgraph" in ds: return 0.45127*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-2500_MA0-800_13TeV-madgraph" in ds: return 0.0007806*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-2500_MA0-700_13TeV-madgraph" in ds: return 0.0008699*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-2500_MA0-600_13TeV-madgraph" in ds: return 0.0009678*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-2500_MA0-500_13TeV-madgraph" in ds: return 0.0010859*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-2500_MA0-400_13TeV-madgraph" in ds: return 0.0013246*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-2500_MA0-300_13TeV-madgraph" in ds: return 0.0025458*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-2000_MA0-800_13TeV-madgraph" in ds: return 0.0020981*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-2000_MA0-700_13TeV-madgraph" in ds: return 0.0024651*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-2000_MA0-600_13TeV-madgraph" in ds: return 0.0028691*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-2000_MA0-500_13TeV-madgraph" in ds: return 0.003341*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-2000_MA0-400_13TeV-madgraph" in ds: return 0.0041934*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-2000_MA0-300_13TeV-madgraph" in ds: return 0.0082317*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1700_MA0-800_13TeV-madgraph" in ds: return 0.0036537*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1700_MA0-700_13TeV-madgraph" in ds: return 0.0045823*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1700_MA0-600_13TeV-madgraph" in ds: return 0.0056149*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1700_MA0-500_13TeV-madgraph" in ds: return 0.0068221*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1700_MA0-400_13TeV-madgraph" in ds: return 0.0088466*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1700_MA0-300_13TeV-madgraph" in ds: return 0.017786*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1400_MA0-800_13TeV-madgraph" in ds: return 0.0055221*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1400_MA0-700_13TeV-madgraph" in ds: return 0.0079659*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1400_MA0-600_13TeV-madgraph" in ds: return 0.010822*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1400_MA0-500_13TeV-madgraph" in ds: return 0.014225*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1400_MA0-400_13TeV-madgraph" in ds: return 0.019609*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1400_MA0-300_13TeV-madgraph" in ds: return 0.041208*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1200_MA0-800_13TeV-madgraph" in ds: return 0.005655*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1200_MA0-700_13TeV-madgraph" in ds: return 0.010028*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1200_MA0-600_13TeV-madgraph" in ds: return 0.015763*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1200_MA0-500_13TeV-madgraph" in ds: return 0.022935*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1200_MA0-400_13TeV-madgraph" in ds: return 0.034039*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1200_MA0-300_13TeV-madgraph" in ds: return 0.075451*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1000_MA0-800_13TeV-madgraph" in ds: return 0.0027483*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1000_MA0-700_13TeV-madgraph" in ds: return 0.0085776*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1000_MA0-600_13TeV-madgraph" in ds: return 0.01897*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1000_MA0-500_13TeV-madgraph" in ds: return 0.03431*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1000_MA0-400_13TeV-madgraph" in ds: return 0.058824*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1000_MA0-300_13TeV-madgraph" in ds: return 0.14383*0.578
        if "MonoHbb_ZpBaryonic_MZp-10_MChi-1_13TeV-madgraph" in ds: return 2.594752001*0.578
        if "MonoHbb_ZpBaryonic_MZp-10_MChi-10_13TeV-madgraph" in ds: return 0.0119285485*0.578
        if "MonoHbb_ZpBaryonic_MZp-10_MChi-50_13TeV-madgraph" in ds: return 0.00020487139962*0.578
        if "MonoHbb_ZpBaryonic_MZp-10_MChi-150_13TeV-madgraph" in ds: return 3.14857697501e-06*0.578
        if "MonoHbb_ZpBaryonic_MZp-10_MChi-500_13TeV-madgraph" in ds: return 4.5071850285e-09*0.578
        if "MonoHbb_ZpBaryonic_MZp-10_MChi-1000_13TeV-madgraph" in ds: return 2.7728742863e-11*0.578
        if "MonoHbb_ZpBaryonic_MZp-15_MChi-10_13TeV-madgraph" in ds: return 0.041809924451*0.578
        if "MonoHbb_ZpBaryonic_MZp-20_MChi-1_13TeV-madgraph" in ds: return 2.72240963862*0.578
        if "MonoHbb_ZpBaryonic_MZp-50_MChi-1_13TeV-madgraph" in ds: return 3.25607022875*0.578
        if "MonoHbb_ZpBaryonic_MZp-50_MChi-10_13TeV-madgraph" in ds: return 3.23855677828*0.578
        if "MonoHbb_ZpBaryonic_MZp-50_MChi-50_13TeV-madgraph" in ds: return 0.00718287240877*0.578
        if "MonoHbb_ZpBaryonic_MZp-95_MChi-50_13TeV-madgraph" in ds: return 0.185594698773*0.578
        if "MonoHbb_ZpBaryonic_MZp-100_MChi-1_13TeV-madgraph" in ds: return 3.18024514561*0.578
        if "MonoHbb_ZpBaryonic_MZp-100_MChi-10_13TeV-madgraph" in ds: return 3.17354850148*0.578
        if "MonoHbb_ZpBaryonic_MZp-200_MChi-1_13TeV-madgraph" in ds: return 2.5517404181*0.578
        if "MonoHbb_ZpBaryonic_MZp-200_MChi-50_13TeV-madgraph" in ds: return 2.09243596445*0.578
        if "MonoHbb_ZpBaryonic_MZp-200_MChi-150_13TeV-madgraph" in ds: return 0.00318936616712*0.578
        if "MonoHbb_ZpBaryonic_MZp-295_MChi-150_13TeV-madgraph" in ds: return 0.133893039105*0.578
        if "MonoHbb_ZpBaryonic_MZp-300_MChi-1_13TeV-madgraph" in ds: return 2.26717461494*0.578
        if "MonoHbb_ZpBaryonic_MZp-300_MChi-50_13TeV-madgraph" in ds: return 1.94713513166*0.578
        if "MonoHbb_ZpBaryonic_MZp-500_MChi-1_13TeV-madgraph" in ds: return 1.09367084934*0.578
        if "MonoHbb_ZpBaryonic_MZp-500_MChi-150_13TeV-madgraph" in ds: return 0.679726638856*0.578
        if "MonoHbb_ZpBaryonic_MZp-500_MChi-500_13TeV-madgraph" in ds: return 2.05703477766e-05*0.578
        if "MonoHbb_ZpBaryonic_MZp-995_MChi-500_13TeV-madgraph" in ds: return 0.0111215826866*0.578
        if "MonoHbb_ZpBaryonic_MZp-1000_MChi-1_13TeV-madgraph" in ds: return 0.201769753605*0.578
        if "MonoHbb_ZpBaryonic_MZp-1000_MChi-150_13TeV-madgraph" in ds: return 0.19601065092*0.578
        if "MonoHbb_ZpBaryonic_MZp-1000_MChi-1000_13TeV-madgraph" in ds: return 5.51545710471e-07*0.578
        if "MonoHbb_ZpBaryonic_MZp-1995_MChi-1000_13TeV-madgraph" in ds: return 0.000729689653045*0.578
        if "MonoHbb_ZpBaryonic_MZp-2000_MChi-1_13TeV-madgraph" in ds: return 0.0139348225665*0.578
        if "MonoHbb_ZpBaryonic_MZp-2000_MChi-500_13TeV-madgraph" in ds: return 0.012671852392*0.578
        if "MonoHbb_ZpBaryonic_MZp-10000_MChi-1_13TeV-madgraph" in ds: return 1.19161364309e-08*0.578
        if "MonoHbb_ZpBaryonic_MZp-10000_MChi-10_13TeV-madgraph" in ds: return 1.19314052769e-08*0.578
        if "MonoHbb_ZpBaryonic_MZp-10000_MChi-50_13TeV-madgraph" in ds: return 1.18388857948e-08*0.578
        if "MonoHbb_ZpBaryonic_MZp-10000_MChi-150_13TeV-madgraph" in ds: return 1.15848042141e-08*0.578
        if "MonoHbb_ZpBaryonic_MZp-10000_MChi-500_13TeV-madgraph" in ds: return 9.56417074962e-09*0.578
        if "MonoHbb_ZpBaryonic_MZp-10000_MChi-1000_13TeV-madgraph" in ds: return 6.59793071766e-09*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-2750_MA0-300_13TeV-madgraph" in ds: return 0.0015511*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-2750_MA0-500_13TeV-madgraph" in ds: return 0.00067388*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-2750_MA0-800_13TeV-madgraph" in ds: return 0.00050351*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-3000_MA0-300_13TeV-madgraph" in ds: return 0.00092077*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-3000_MA0-500_13TeV-madgraph" in ds: return 0.0004022*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-3000_MA0-800_13TeV-madgraph" in ds: return 0.00030894*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-3500_MA0-300_13TeV-madgraph" in ds: return 0.00033718*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-3500_MA0-500_13TeV-madgraph" in ds: return 0.00014885*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-3500_MA0-800_13TeV-madgraph" in ds: return 0.00011855*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-4000_MA0-300_13TeV-madgraph" in ds: return 0.00012964*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-4000_MA0-500_13TeV-madgraph" in ds: return 5.7149e-05*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-4000_MA0-800_13TeV-madgraph" in ds: return 4.6312e-05*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-800_MA0-600_13TeV-madgraph" in ds: return 0.010350*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-800_MA0-500_13TeV-madgraph" in ds: return 0.037616*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-800_MA0-400_13TeV-madgraph" in ds: return 0.093376*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-800_MA0-300_13TeV-madgraph" in ds: return 0.2848*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-600_MA0-400_13TeV-madgraph" in ds: return 0.065307*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-600_MA0-300_13TeV-madgraph" in ds: return 0.46223*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-2500_MA0-800_13TeV-madgraph" in ds: return 0.00082449*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-2500_MA0-700_13TeV-madgraph" in ds: return 0.00091718*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-2500_MA0-600_13TeV-madgraph" in ds: return 0.0010187*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-2500_MA0-500_13TeV-madgraph" in ds: return 0.0011446*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-2500_MA0-400_13TeV-madgraph" in ds: return 0.0013922*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-2500_MA0-300_13TeV-madgraph" in ds: return 0.0026787*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-2000_MA0-800_13TeV-madgraph" in ds: return 0.0021931*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-2000_MA0-700_13TeV-madgraph" in ds: return 0.0025816*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-2000_MA0-600_13TeV-madgraph" in ds: return 0.0029991*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-2000_MA0-500_13TeV-madgraph" in ds: return 0.0034998*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-2000_MA0-400_13TeV-madgraph" in ds: return 0.0043899*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-2000_MA0-300_13TeV-madgraph" in ds: return 0.0086008*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1700_MA0-800_13TeV-madgraph" in ds: return 0.0038043*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1700_MA0-700_13TeV-madgraph" in ds: return 0.0047758*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1700_MA0-600_13TeV-madgraph" in ds: return 0.0058468*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1700_MA0-500_13TeV-madgraph" in ds: return 0.0071071*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1700_MA0-400_13TeV-madgraph" in ds: return 0.0092082*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1700_MA0-300_13TeV-madgraph" in ds: return 0.018545*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1400_MA0-800_13TeV-madgraph" in ds: return 0.0057179*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1400_MA0-700_13TeV-madgraph" in ds: return 0.0082681*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1400_MA0-600_13TeV-madgraph" in ds: return 0.011224*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1400_MA0-500_13TeV-madgraph" in ds: return 0.014723*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1400_MA0-400_13TeV-madgraph" in ds: return 0.020306*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1400_MA0-300_13TeV-madgraph" in ds: return 0.042687*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1200_MA0-800_13TeV-madgraph" in ds: return 0.0058353*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1200_MA0-700_13TeV-madgraph" in ds: return 0.010364*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1200_MA0-600_13TeV-madgraph" in ds: return 0.016254*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1200_MA0-500_13TeV-madgraph" in ds: return 0.023706*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1200_MA0-400_13TeV-madgraph" in ds: return 0.03524*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1200_MA0-300_13TeV-madgraph" in ds: return 0.078164*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1000_MA0-800_13TeV-madgraph" in ds: return 0.00283922*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1000_MA0-700_13TeV-madgraph" in ds: return 0.00881814*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1000_MA0-600_13TeV-madgraph" in ds: return 0.019542*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1000_MA0-500_13TeV-madgraph" in ds: return 0.035347*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1000_MA0-400_13TeV-madgraph" in ds: return 0.060531*0.578
        if "ZprimeToA0hToA0chichihbb_2HDM_MZp-1000_MA0-300_13TeV-madgraph" in ds: return 0.14771*0.578

        return 1.0


datasets = [(ds, category(ds)) for ds in bg_datasets + signal_samples]
luminosities = [(ds,luminosity(ds)) for ds in  bg_datasets]



#for ds, cat in datasets: print ds, cat
#for ds, lum in luminosities: print ds, lum
    
test_dataset =[["ZprimeToA0hToA0chichihbb_2HDM_MZp-600_MA0-300_13TeV-madgraph","a"],[category("ZprimeToA0hToA0chichihbb_2HDM_MZp-600_MA0-300_13TeV-madgraph"),"b"]]



#test_dataset = [ds.strip() for ds in test_dataset if ds.strip]
session = Session(job_server)

In [3]:
#__worker_class__

import numpy as np

def p4_pt(p4):
    # p4 is [n,4] numpy array
    # returns [n] numpy array
    return np.sqrt(p4[:,0]**2 + p4[:,1]**2)

def p4_eta(p4):
    p42 = p4**2
    p3 = np.sqrt(np.sum(p42[:,:3], axis=-1))
    return p4[:,2]/p3

def valueAt(dictionary ,key,x,y = None):
    if y ==None:
        if not isinstance(dictionary[key][1][0], np.float64):
            print "dictionary[" +key+"] is  2D"
            return "dictionary[" +key+"] is  2D"
        xBinMin = dictionary[key][1][0]
        xBinMax = dictionary[key][1][len(dictionary[key][1])-2]
        if x<xBinMin or x> xBinMax:
            print "point out  of range" + " ("+str(xBinMin) + ","+ str(xBinMax) +")"
            return 1
        i = 0

        xBin = dictionary[key][1][0]


        while(x>xBin and x < len(dictionary[key][1])-1):
            i = i +1
            xBin = dictionary[key][1][i]

        i = i-1
        
        return dictionary[key][0][i] 
    else:
    
        if isinstance(dictionary[key][1][0], np.float64):
           # print "dictionary[" +key+"] is not 2D"
            return "dictionary[" +key+"] is not 2D"
        xBinMin = dictionary[key][1][0][0]
        xBinMax = dictionary[key][1][0][len(dictionary[key][1][0])-1]
        yBinMin = dictionary[key][1][1][0]
        yBinMax = dictionary[key][1][1][len(dictionary[key][1][1])-1]
        #if x<xBinMin or x> xBinMax or y < yBinMin or y > yBinMax:
            #print "point out  of range x:("+str(xBinMin)+","+str(xBinMax)+") y:("+str(yBinMin)+","+str(yBinMax) +")"
            #return x,xBinMin,xBinMax,"aaa",y,yBinMin,yBinMax#"point out  of range x:("+str(xBinMin)+","+str(xBinMax)+") y:("+str(yBinMin)+","+str(yBinMax) +")"
            
            #return 1
        i = 0
        j = 0
        xBin = dictionary[key][1][0][0]
        yBin = dictionary[key][1][1][0]
        
        while(x>xBin and i< len(dictionary[key][1][0])-2):
            i = i +1
            xBin = dictionary[key][1][0][i]
        while(y>yBin and j < len(dictionary[key][1][1])-2):
            j = j +1
            yBin = dictionary[key][1][1][j]
        i = i-1
        j = j-1
        if x <=xBinMin:
            i = 0
        if x >=xBinMax:
            i = len(dictionary[key][1][0])-2
        if y < yBinMin:
            j = 0
        if y >= yBinMax:
            j = len(dictionary[key][1][1])-2
        return dictionary[key][0][j][i]

def makeSfSpecificVec(csvVec,OP, mType,sysType, flavor):
    OPVec = []
    for i,e in enumerate(csvVec):
        if e[0] == OP and e[1] ==mType and e[2]==sysType and e[3] == flavor:
            e.pop(0)
            e.pop(0)
            e.pop(0)
            e.pop(0)
            OPVec.append(e)
    
    return OPVec

def calcSFBtag(sfCalcVec, eta, pt):
    ret = 0
    if pt < sfCalcVec[0][2]:
        pt = sfCalcVec[0][2]+0.0001
    if pt > sfCalcVec[len(sfCalcVec)-1][3]:
        pt = sfCalcVec[len(sfCalcVec)-1][3] -0.0001
    for e in sfCalcVec:
        if pt > e[2] and pt < e[3]:
            x = pt
            ret = eval(e[6])
            return ret
class Worker(object):
    '''
    Columns = ["Weight", "NMuons", "HT", "MHT", "NElectrons", "NJets", "BTags",
            "isoElectronTracks", "isoMuonTracks", "isoPionTracks",
            "DeltaPhi1", "DeltaPhi2", "DeltaPhi3", "DeltaPhi4",
            "JetID", "PFCaloMETRatio","Trigger.pass",
            "NVtx", "eeBadScFilter", "HBHENoiseFilter",
            "HBHEIsoNoiseFilter", "globalTightHalo2016Filter",
            "EcalDeadCellTriggerPrimitiveFilter", "BadChargedCandidateFilter",
            "BadPFMuonFilter",
             "Jet.p4"]
    '''
    Columns = ["weight","muons.pt_","muons.eta_","muons_size", "npvTrue","muons.loose",
               "muons.tight", "electrons.loose", "electrons.veto","electrons.tight",
              "electrons.pt_", "electrons.eta_", "electrons_size","ak4GenJets.eta_","ak4GenJets.pt_"]
    def run(self, events, job):
        
        category = job["category"]
        lumi = job["lumi"]
        xsec_weight = lumi*events.weight/9000
        corrDict = job["corrDict"]
        csvVec = job["csvVec"]
        job.message(str(csvVec[0]))
        #job.message(str(makeSfSpecificVec(csvVec,0,'mujets','central',0)))
        sfTable=makeSfSpecificVec(csvVec,0,'mujets','central',0)
        job.message(str(sfTable))
        #job.message(str(OPVec[0]))
        job.message(str(events.ak4GenJets.pt_.size))
        jets_filter = events.filter(events.ak4GenJets.count > 0)
        events = jets_filter(events)                # filter out events without jets
        sfTemp = []
        sfTot = []
        for i, e in enumerate(events):
            jets = e.ak4GenJets
            sfVec=[]
            for j in jets:
                sfVec.append(calcSFBtag(sfTable,j.eta_ ,j.pt_))
                sfTot.append(calcSFBtag(sfTable,j.eta_ ,j.pt_))
            sfTemp.append(sfVec)
        n=np.array(sfTemp)
        job.message(str(n.size))
      #  job.message(str(type( events.ak4GenJets.pt_[0])))
        events.ak4GenJets.sf = np.array(sfTot)
        for e in events:
            job.message(str(e.ak4GenJets.sf))
        j=0
        for i in events.muons_size:
            j = j +i
        #for ie, e in enumerate(events):
         #   for pt
       # job.send(a=xsec_weight)
        # Weight calculation i
        sf_pu = [valueAt(corrDict,"puWeights_80x_37ifb_puWeights",i) for i in events.npvTrue]
        sf_lepID = np.array([])
        #job.message(str(sf_pu))
        #job.message(str(len(events.muons.pt_)))
        for ie,e in enumerate(events):
            sf = 1
            ePt = e.electrons.pt_
            eEta = e.electrons.eta_
            eT = e.electrons.tight
            eType = [11 for x in e.electrons.pt_]
            
            electrons = zip(ePt,eEta,eT,eType)
        
            mPt = e.muons.pt_
            mEta = e.muons.eta_
            mT = e.muons.tight
            mType = [13 for x in e.muons.pt_]
            
            muons = zip(mPt,mEta,mT,mType)
            if not ePt.size == 0 and  mPt.size ==0:
                leptons = []
            elif  ePt.size==0:
                leptons = muons
            elif  mPt.size ==0:
                leptons = electrons
            else:
                leptons = (electrons+muons)
            leptons = sorted(leptons)[::-1]
            #job.message(str(leptons))
            for i in range(0, int(np.minimum(2,len(leptons)))):
                if leptons[i][3] ==11:
                    if abs(leptons[i][1]) <2.5 and leptons[i][0] > 10:
                        if leptons[i][2]:
                            #job.message(str(valueAt(corrDict,"scalefactors_80x_egpog_37ifb_scalefactors_Tight_Electron",e.electrons.eta_[i],e.electrons.pt_[i])))
                            sf = sf *valueAt(corrDict,"scalefactors_80x_egpog_37ifb_scalefactors_Tight_Electron",leptons[i][1], leptons[i][0])

                        else:
                            #job.message(str(valueAt(corrDict,"scaleFactor_electron_summer16_scaleFactor_electron_vetoid_RooCMSShape_pu_0_100",e.electrons.eta_[i],e.electrons.pt_[i])))
                            sf = sf *valueAt(corrDict,"scaleFactor_electron_summer16_scaleFactor_electron_vetoid_RooCMSShape_pu_0_100",leptons[i][1], leptons[i][0])

                if leptons[i][3]==13:
                    if abs(leptons[i][1]) <2.4 and leptons[i][0] > 10:
                        if leptons[i][2]:
                            #job.message(str(valueAt(corrDict,"leptonic_muon_scalefactors_37ifb_scalefactors_TightId_Muon",abs(e.muons.eta_[i]),e.muons.pt_[i])))
                            sf = sf *valueAt(corrDict,"leptonic_muon_scalefactors_37ifb_scalefactors_TightId_Muon",abs(leptons[i][1]), leptons[i][0])

                        else:
                            #job.message(str(valueAt(corrDict,"leptonic_muon_scalefactors_37ifb_scalefactors_MuonLooseId_Muon",abs(e.muons.eta_[int(i)]),e.muons.pt_[int(i)])))
                            sf=sf*valueAt(corrDict,"leptonic_muon_scalefactors_37ifb_scalefactors_MuonLooseId_Muon",abs(leptons[i][1]), leptons[i][0])

            sf_lepID = np.append(sf_lepID,sf) 
            #job.message(str(sf_lepID))
        '''
        
        basic_cuts = events.filter((events.NJets >= 2) * (events.HT > 300) * (events.MHT>300)
            * (events.NMuons == 0) * (events.NElectrons == 0) * (events.isoElectronTracks==0)
            * (events.isoMuonTracks==0) * (events.isoPionTracks==0) 
            * (events.DeltaPhi1 > 0.5) * (events.DeltaPhi2 > 0.5) 
            * (events.DeltaPhi3 > 0.3) * (events.DeltaPhi4 > 0.3)
            * (events.JetID==1) * (events.PFCaloMETRatio < 5) * (events.NVtx > 0)
            * (events.eeBadScFilter==1)
            * (events.HBHENoiseFilter==1)
            * (events.HBHEIsoNoiseFilter==1)
            * (events.globalTightHalo2016Filter==1)def valueAt(dictionary ,key,x,y = None):
    

            * (events.EcalDeadCellTriggerPrimitiveFilter==1)
            * (events.BadChargedCandidateFilter==1)
            * (events.BadPFMuonFilter==1))
      
        job.fill(category=category, basic_cut_acceptance=[basic_cuts.ratio])
        
        events = basic_cuts(events)
        '''
        '''
      # HT5/HT filter
        jets = events.Jet
        jets_pt = p4_pt(jets.p4)
        jets_eta = p4_eta(jets.p4)
        jet_filter_out = (jets_pt <= 30) + (jets_eta >= 5)
        pt_good = jets_pt.copy()
        pt_good[jet_filter_out] = 0.0
        jets.pt_good = pt_good
        ht5 = np.empty_like(events.HT)
        trigger_filter = events.filter()
        for ie, e in enumerate(events):
            
            #if basic_cuts.Mask[ie]:
            
                # ht5 calculation
                ejets = e.Jet
                ht5[ie] = sum([j.pt_good for j in ejets])

                # good trigger
                # can not write "e.Trigger[i].pass" because "pass" is reserved word in Python 
                trigger_filter.Mask[ie] = np.any([e.Trigger[i].dot("pass") for i in [53, 54, 55, 57, 58, 59]])
            
        ht5ht_filter = events.filter((ht5/np.maximum(events.HT, 1.0) <= 2.0))
        
        final_filter = trigger_filter * ht5ht_filter
        
        final_events = final_filter(events)
        '''
        job.fill(
            category=category,
            NJets = 0,#final_events.NJets,
            #BTags = 0 #final_events.BTags,
          #  HT = 0# final_events.HT,
           # MHT = 0#final_events.MHT,
            weight = events.weight
            #final_events.Weight
            
        )
        
        

In [4]:
basic_cut_narrow = Hist(hbin("basic_cut_acceptance", 50, 0, 0.1), groupby("category", keeporder=True))
basic_cut_wide = Hist(hbin("basic_cut_acceptance", 50, 0, 1.0), groupby("category", keeporder=True))

NJets = Hist(hbin("NJets", 20, 0, 20), groupby("category", keeporder=True), weight="weight")
BTags = Hist(hbin("BTags", 10, 0, 10), groupby("category", keeporder=True), weight="weight")
HT = Hist(hbin("HT", 50, 300, 1800), groupby("category", keeporder=True), weight="weight")
MHT = Hist(hbin("MHT", 50, 300, 1300), groupby("category", keeporder=True), weight="weight")

display = IPythonDisplay(
    below(
        beside(
            basic_cut_narrow.overlay("category").step("basic_cut_acceptance", yscale={"type":"log"}),
            basic_cut_wide.overlay("category").step("basic_cut_acceptance", yscale={"type":"log"})
        ),
        beside(
            NJets.overlay("category").step("NJets"),
            BTags.overlay("category").step("BTags")
        ),
        beside(
            HT.overlay("category").step("HT"),
            MHT.overlay("category").step("MHT")
        )
    )
)

class Callback:
    
    def __init__(self, display):
        self.Display = display
        
    def on_histogram_update(self, nevents):
        self.Display.update()
    def on_stream_update(self,nEvents, data):
        for x in data["a"]:
            print x
    def on_message(self,workerId, nEvents, message):
        print message
    def on_exception(self,workerId, info):
        print info
        

In [ ]:
display.init()
callback = Callback(display)

import time
nevents_total = 0
t1 = time.time()
for dataset, category in test_dataset:
    lumi = luminosity(test_dataset)
    job = session.createJob(dataset, 
            fraction= 0.2,                   # look at 20% events from each dataset
            user_callback=callback, 
            user_params = {"category":category, "lumi":lumi, "corrDict":corrDict,"csvVec":csvVec},
            histograms = [basic_cut_narrow, basic_cut_wide, NJets, BTags, HT, MHT]
    )
   
    job.run()
    runtime = job.TFinish - job.TStart
    nevents = job.EventsProcessed
    nevents_total += nevents
   # print "%-70s %7.3f M events, %7.3f M events/sec" % (dataset[:70], float(nevents)/1000000, nevents/runtime/1000000)
    #print "%s: %.6f million events/second" % (dataset, nevents/runtime/1000000)
   # display.update()

t2 = time.time()
print "Total events processed: %d in %.1f seconds -> %.6f million events/second" %(nevents_total, t2-t1, nevents_total/(t2-t1)/1000000)


[0, 'mujets', 'central', 1, -2.4, 2.4, 20.0, 1000.0, 0.0, 1.0, '0.884016*((1.+(0.0331508*x))/(1.+(0.0285096*x)))']
[[-2.4, 2.4, 20.0, 1000.0, 0.0, 1.0, '0.884016*((1.+(0.0331508*x))/(1.+(0.0285096*x)))']]
46787
9000
[ 1.00284749  1.0006783   0.98425111  0.94378275]
[ 1.00521581  0.98615123  0.98326184  0.98234194  0.96740711  0.93875098]
[ 1.00472327  0.99118873  0.98784216  0.959568  ]
[ 1.00111089  0.99859218  0.97338522  0.94090079]
[ 1.00407373  0.97910044  0.97038945  0.97011603  0.95127488  0.93627595]
[ 1.00662917  1.00051462  0.98978167  0.97580619  0.9740648 ]
[ 1.00344784  0.99795219  0.99130223  0.96524802  0.93928022  0.93627595
  0.93627595]
[ 1.00026372  0.9941592   0.98469994  0.95759324  0.93627595]
[ 1.00803946  1.000091    0.99539128  0.99222305  0.98383094]
[ 1.00410309  0.99919974  0.97746433  0.97586472]
[ 1.00816823  1.00280279  0.99707836  0.94243994]
[ 0.99822627  0.98425859  0.97112723  0.95224336  0.94826949]
[ 1.00029263  0.99011694  0.98624319  0.98340442  0

[ 1.00380372  0.99798198  0.99790249  0.97690066]
[ 1.00642491  1.0026069   1.00009289  0.95071606]
[ 1.00216834  0.99363948  0.9884382   0.98638272  0.97728502  0.95977464
  0.93627595]
[ 1.00033996  0.99441111  0.98880206  0.98588291]
[ 1.00808443  0.99526683  0.9748634   0.94156973]
[ 1.00380791  0.99510594  0.99260399  0.98798196]
[ 1.00551109  1.00488305  0.98755586]
[ 0.99926495  0.99342601  0.97733275  0.94838236]
[ 1.00880958  1.00317238  0.99254656  0.98569382  0.98030985  0.96156707
  0.93643251  0.93627595]
[ 1.00740265  1.00280452  0.98087247  0.97792038  0.97055944  0.95101686
  0.93627595]
[ 1.00635257  0.98338749  0.9809344   0.97251366  0.96257026]
[ 1.00879632  0.99525542  0.97840596  0.9720936   0.93627595  0.93627595
  0.93627595]
[ 0.99511048  0.99444569  0.99323954  0.98633509  0.97917242]
[ 1.00743343  1.00245476  0.98789483  0.9623158   0.94212363  0.93627595]
[ 1.00890252  1.00822771  1.00746008  0.98367874  0.9637334   0.95359725
  0.93912358  0.93772325]
[ 1.0

[ 1.00254466  1.00123786  0.99263233  0.97428964  0.94038279  0.93826358
  0.93627595]
[ 1.00480174  1.00210558  0.96389964  0.94092264]
[ 1.00401286  0.99522216  0.97174161  0.94702858]
[ 1.00627155  1.00217314  0.95702893  0.95472138]
[ 1.0049724   1.00345449  0.96082445  0.93627595]
[ 0.99606365  0.97322958  0.9581584   0.95286379  0.95285854  0.94647776
  0.93627595]
[ 1.00437435  1.00309496  0.98791197  0.96840453  0.95710341]
[ 1.00962117  0.99519617  0.97921851  0.93991565]
[ 0.99694939  0.9953989   0.99533752  0.97809584]
[ 1.00903915  0.99539859  0.98367795  0.98260231  0.96650123]
[ 0.99836742  0.98691474  0.97025964  0.95778335]
[ 1.00791071  0.9983545   0.9911331   0.95641803  0.93627595  0.93627595
  0.93627595]
[ 0.99010236  0.98070201  0.97849088  0.97535428  0.94373444]
[ 1.00733441  0.98835186  0.96860616  0.94146387  0.93627595]
[ 1.00793172  1.00233867  0.99524268  0.9938797   0.96144696  0.96103456
  0.9555393   0.93627595]
[ 1.00727983  0.99952077  0.98462471  0.95

[ 0.99903258  0.97990513  0.97874913  0.96448652  0.93627595]
[ 1.00781885  0.99539454  0.98452456]
[ 1.00663094  1.00061265  0.96136272  0.93837939]
[ 1.00334304  1.00332121  0.97687883  0.96964049  0.93627595  0.93627595]
[ 1.00749499  1.00136252  0.99900426]
[ 1.01549834  1.01492596  0.97225024  0.96931784  0.96841776  0.9382358 ]
[ 1.00699741  1.00359773  0.98367108  0.96226567]
[ 1.0033401   1.00202898  0.99122649  0.94039178]
[ 1.0059074   1.00380119  0.99254848  0.95007552  0.93627595]
[ 0.99733141  0.99405943  0.99344132  0.98956822  0.93627595]
[ 0.99988273  0.98197193  0.97906845  0.96623901]
[ 1.00533494  0.99486503  0.99166145  0.98770462  0.97198556  0.97119816
  0.96361386]
[ 1.00949917  1.00906843  0.99648315  0.99389896  0.98019499  0.96186974
  0.94913906]
[ 1.00900158  1.00185083  0.95114122  0.93627595]
[ 1.00890032  1.00527261  0.96668505]
[ 1.00944232  0.9960135   0.98183436  0.96708822  0.96469759  0.94434343
  0.93654229]
[ 1.00275948  0.99888911  0.99228938  0.9

[ 1.00369297  0.99525714  0.98897486  0.9875407   0.97468538  0.96532704
  0.95117964  0.9458579   0.93879767  0.93627595  0.93627595]
[ 1.00698718  0.99653328  0.98521332  0.9847791 ]
[ 1.00647766  0.99662755  0.96862397  0.96605545  0.96272199]
[ 1.00300502  0.99672714  0.99048402  0.98428965]
[ 1.00541194  0.99800461  0.99389612  0.98204383  0.9582232   0.94178222]
[ 1.00416822  1.00380903  0.99102319  0.97169927  0.97034253]
[ 1.00220937  1.00025244  0.97496668  0.94305824]
[ 1.00411448  1.00106888  0.99975578  0.98012803  0.93944964]
[ 1.00306122  0.99964738  0.98097511  0.96952389  0.93627595]
[ 0.99250851  0.99229136  0.99050552  0.98498422  0.94611166]
[ 1.00568713  0.99936133  0.98779863  0.97974463]
[ 1.00692382  1.00358063  0.95415949  0.94064845]
[ 1.0051797   1.00377579  0.98809571  0.9640877   0.93627595  0.93627595]
[ 1.00672786  0.98484956  0.98455755  0.96730898]
[ 1.01219209  1.00955486  0.99242512  0.98832601  0.98073186  0.95806413
  0.94515571  0.94425378]
[ 1.0093

...

[ 1.0072578   1.00446307  0.97695752  0.96730026  0.96407982]
[ 1.00483676  0.99621253  0.99384523  0.99346353]
[ 1.00601687  0.99031992  0.98466592  0.95093366  0.93627595]
[ 0.99801312  0.99038097  0.98414034  0.97563514  0.94664429]
[ 1.00406783  1.00169241  0.9751388   0.96786005  0.94604557]
[ 1.01063847  1.00667935  0.9947157   0.99094226  0.95222702]
[ 1.00765326  1.00363315  0.96047753  0.94773986]
[ 1.00126728  0.99122397  0.988736    0.9842101   0.94312623]
[ 1.0093428   1.00145954  0.9934028   0.97686758  0.96772085  0.94427143
  0.93869098  0.93627595  0.93627595  0.93627595]
[ 1.01016255  1.00020853  0.96103782  0.95901703  0.93627595]
[ 1.00496967  1.00168322  0.99115495  0.97125842  0.94671729  0.93627595]
[ 1.00074897  0.99266942  0.98749413  0.98438512]
[ 1.00438577  0.99632212  0.99435129  0.99369985  0.98801598  0.94232892
  0.94097942  0.93764877]
[ 1.00175168  1.00074672  0.9907373   0.97842957  0.946678    0.93627595]
[ 1.00416953  0.99897017  0.98353357]
[ 1.0000

[ 1.00677476  1.00676117  0.96564629  0.95604034  0.93877036]
[ 1.0089263   1.00393006  0.95935922  0.95163402  0.93627595]
[ 1.00789624  0.99746195  0.98455103  0.95355821]
[ 1.00352952  0.99688099  0.98594314  0.97046088  0.93949193  0.93627595
  0.93627595]
[ 0.99280832  0.99256308  0.99202833  0.98864542  0.95323995]
[ 1.00387025  1.00375374  0.99841169  0.99533953  0.96191645  0.93919663
  0.93627595  0.93627595]
[ 0.99846687  0.99801155  0.97705591  0.9546749   0.93627595]
[ 1.00395465  1.00098583  0.99610516  0.97592922  0.94493904  0.93627595]
[ 1.00840301  0.99228481  0.97319971  0.96699174  0.96413264]
[ 1.00893488  0.99358093  0.97895283  0.96478716  0.94198013  0.93627595]
[ 1.0017214   0.99230592  0.98321047  0.97253738  0.95202068]
[ 1.00555773  1.00276881  0.97821676]
[ 1.0062869   0.99897145  0.96032745  0.9554663   0.95335635]
[ 1.00822282  0.99552419  0.98324059  0.98095246  0.95439953]
[ 1.00627531  0.99620983  0.99441452  0.93627595]
[ 1.00492725  1.00441168  0.9864

[ 1.01180052  1.00283602  0.97406505  0.9699634   0.95435871  0.93627595]
[ 1.0065147   1.00302525  0.99840701  0.97723972  0.93627595]
[ 1.00554595  1.00237512  0.98398223  0.98288671  0.97818742  0.96612285
  0.95594888  0.93627595]
[ 0.99574786  0.99165912  0.97562214  0.97189086  0.96388688  0.93837295]
[ 1.00596253  1.00275445  1.00115765  0.99989519  0.97449704]
[ 1.00606905  0.99115434  0.986809    0.9599763 ]
[ 1.00678072  1.00500476  0.98324799  0.94239411]
[ 0.99898775  0.99829874  0.98592323  0.97975231  0.95090276  0.93627595
  0.93627595]
[ 1.01465691  1.00709325  0.9955272   0.99083726  0.97430542  0.95198154
  0.9473968   0.93627595  0.93627595]
[ 1.00193555  0.99262376  0.95264839  0.94397205]
[ 1.00382462  0.99802528  0.97479304  0.94000458  0.93845258  0.93627595]
[ 1.00380843  1.00304726  0.98045341  0.95303961  0.95207494  0.94707122
  0.94687589]
[ 1.00207706  1.00200391  0.9868435   0.9783685   0.93627595]
[ 1.00489669  1.00176784  0.99187384  0.9753507   0.936275

[ 1.00807806  1.00402651  1.00214147  0.98702455  0.96331842  0.95455556]
[ 1.00131055  0.98114037  0.97795085  0.9557648 ]
[ 0.99600679  0.99223948  0.98620599  0.95596612  0.95261905]
[ 1.00725142  0.98996428  0.98776422  0.97032218]
[ 1.00637856  0.98891477  0.97921258  0.96624086]
[ 0.99469415  0.98596137  0.98235585  0.97203524]
[ 1.00567344  0.99912652  0.981773    0.9631658 ]
[ 0.99779486  0.99725997  0.98947004  0.96832272  0.96043619]
[ 1.00907848  1.00065931  0.99279057]
[ 1.00477374  1.00245609  0.99089937  0.95647649  0.94429321]
[ 0.99996584  0.99740504  0.99563639  0.9861783 ]
[ 1.00722134  1.00012448  0.98182733  0.97601445]
[ 1.00289219  0.99277768  0.98321328  0.94708813  0.93627595  0.93627595]
[ 0.99654181  0.99083557  0.96522089  0.9416765   0.93627595]
[ 1.00308382  0.99826709  0.98761639  0.9468796 ]
[ 1.00447249  0.98080529  0.97265436  0.95068277  0.93627595]
[ 1.00287379  0.99594605  0.98466724  0.98249605  0.97249879  0.95616413
  0.95326271  0.94645285  0.936

[ 1.00720205  0.99974689  0.97503723  0.96123608  0.95926708  0.95366393]
[ 1.00600169  1.00403358  0.99116821  0.94064404  0.93627595]
[ 1.000096    0.99244414  0.98841777  0.98829059]
[ 1.00198979  1.00005676  0.99879666  0.9639365   0.96036819  0.95959085
  0.93823679  0.93627595]
[ 1.00702802  1.00296751  0.96058917  0.95370897  0.95038758  0.94055215
  0.93627595]
[ 1.00705029  0.99335581  0.97196479  0.96349598  0.96117772]
[ 1.00647624  0.99804155  0.99792044  0.93627595  0.93627595]
[ 1.0005462   0.9946495   0.97395374  0.9555863   0.94995399  0.94700525
  0.93938583  0.93627595]
[ 1.0067996   0.98432754  0.98215953  0.97810118  0.93627595]
[ 1.00302974  0.99867492  0.98577404  0.98381823  0.96298723  0.95623573
  0.94031284  0.93627595  0.93627595  0.93627595]
[ 1.0050655   1.00157742  0.97114204  0.93627595]
[ 1.00731988  1.00618514  0.95020784]
[ 1.01372236  1.01278453  1.00704841  0.9739032   0.95274806]
[ 1.00564357  1.00210569  0.98467442  0.96034091  0.94940998]
[ 0.9971